<a href="https://colab.research.google.com/github/pajacques/ElegantRL/blob/master/yes_4.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install --quiet --update transformers


Usage:   
  pip3 install [options] <requirement specifier> [package-index-options] ...
  pip3 install [options] -r <requirements file> [package-index-options] ...
  pip3 install [options] [-e] <vcs project url> ...
  pip3 install [options] [-e] <local project path> ...
  pip3 install [options] <archive url/path> ...

no such option: --update


# 1 - Read text

In [ ]:
!pip install --quiet PyMuPDF

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 19.8/19.8 MB 23.2 MB/s eta 0:00:00


In [ ]:
import sys, pathlib, fitz, os

fname = '/content/Makita - GROUPE LD INC.pdf'
words=""
with fitz.open(fname) as pages:
  for page in pages:
    words += page.get_text("text", sort=True)
    words += "{""SPLIT""}\n"

new_file_path = os.path.splitext(fname)[0] + '_convN' + '.txt'
with open(new_file_path, 'w') as file:
  file.write(words)

words

#Step 2 - Add tags in the text

#2.1 Install libraries

In [ ]:
!pip install --quiet datasets

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 480.6/480.6 kB 8.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 6.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.3/179.3 kB 9.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 6.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 9.2 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2024.10.0 requires fsspec==2024.10.0, but you have fsspec 2024.9.0 which is incompatible.


##2.2 - Loggin onto Hugging Face

In [ ]:
from huggingface_hub import login
from google.colab import userdata

token = userdata.get('HF_TOKEN')

# Log in to Hugging Face
login(token, add_to_git_credential=True)

##2.3 Create Functions to support dataset

In [ ]:
import json
from datasets import Dataset, DatasetDict

data_all = {
            'train': {'text': [], "json":[], "filename":[]},
            'validation': {'text': [], "json":[], "filename":[]},
            'test': {'text': [], "json":[], "filename":[]}
           }

def reset_dataset():
  global data_all

  data_all = {
            'train': {'text': [], "json":[], 'filename':[]},
            'validation': {'text': [], "json":[], 'filename':[]},
            'test': {'text': [], "json":[], 'filename':[]}
           }

def add_to_dataset_straigh(full_text, invoice_str, split="train", filename=''):
    global data_all  # Declare the variable as global to modify it inside the function
    data_all[split]["text"].append(full_text)
    data_all[split]["json"].append(invoice_str)
    data_all[split]["filename"].append(filename)

def push_dataset_to_hf(data_all):
    # Create a Hugging Face DatasetDict from the dictionary with splits
    dataset_dict = DatasetDict({
        'train': Dataset.from_dict(data_all['train']),
        'validation': Dataset.from_dict(data_all['validation']),
        'test': Dataset.from_dict(data_all['test'])
    })
    dataset_dict.push_to_hub('pajacques/yes')

def convert_to_structure(invoice_data):

    price_format = invoice_data.get("sub_total_str", "")

    # Initialize the structure with full_text template placeholders
    data = {
        "metadata": {
            "invoice_number": invoice_data.get("invoice_number_str", ""),
            "purchase_order": invoice_data.get("po_number_str", ""),
            #"date": "N/A",  # Placeholder for date
            "customer": invoice_data.get("customer", ""),
            "lines": [],
            #"freight": "0.00",  # Assuming it's constant
            #"total_tax": "0.00",  # Placeholder for tax
            "gst_number": invoice_data.get("gst_id", ""),
            "gst": format_number_like(invoice_data.get("gst", 0), price_format,"gst-2"),
            "qst_number": invoice_data.get("qst_id", ""),
            "qst": format_number_like(invoice_data.get("qst", 0), price_format,"qst-2"),
            "subtotal": invoice_data.get("sub_total_str", ""),
            "total": invoice_data.get("total_str", ""),
            "currency":invoice_data.get("currency", ""),
        }
    }

    max_value = float('-inf')  # Initialize to negative infinity to find the max
    max_value_format = ""
    max_value_unit_price = float('-inf')
    max_value_unit_price_format = ""


    # Extract the highest total and take note of the value format
    for item in invoice_data['lines']:
      # Ensure 'line_total' exists in the dictionary and is a number
      if 'line_total' in item and isinstance(item['line_total'], (int, float)):
          if item['line_total'] > max_value:
              max_value = item['line_total']
              max_value_format = item['line_total_str']
      if 'unit_price' in item and isinstance(item['unit_price'], (int, float)):
          if item['unit_price'] > max_value_unit_price:
              max_value_unit_price = item['unit_price']
              max_value_unit_price_format = item['unit_price_str']


    # Process each line item
    for index, line in enumerate(invoice_data['lines']):

        quantity = line.get("quantity_str", 0)
        item_code = line.get("item_code", "Unknown")

        # Unit Price
        price = line.get("unit_price", 0)
        price = format_number_like(price, max_value_unit_price_format,"price")

        # Line total
        line_total = line.get("line_total", 0)
        line_total = format_number_like(line_total, max_value_format,"line_total")
        item_description = line.get("item_name", "Unknown")

        # Create the item dictionary for metadata
        item = {
            "item_code": item_code,
            "description": item_description,
            "quantity": quantity,
            "unit_price": price,
            "line_total": line_total
        }

        data["metadata"]["lines"].append(item)

    return data  # Return a list of data to match the structure

#2.4 Analyse text

In [ ]:
import random
import string
import re
import glob
import os

# def get_text_from_line(text, row_number):
#     """
#     Retrieves all the text from a specific line number in the given text.

#     Args:
#         text (str): The multi-line text.
#         row_number (int): The line number to retrieve (1-based index).

#     Returns:
#         str: The text of the specified line, or an error message if the line number is out of range.
#     """
#     # Split the text into lines
#     lines = text.splitlines()

#     # Ensure row_number is within bounds
#     if row_number < 0 or row_number >= len(lines):
#         return "Row number out of range."

#      # Calculate the start and end line indices
#     start = max(0, row_number - 5)
#     end = min(len(lines), row_number + 6)  # +6 because we want to include the current line

#     # Return the lines from start to end
#     return '\n'.join(lines[start:end])

def generate_similar_string(input_string):
    result = []

    for char in input_string:
        if char.isdigit():
            # Randomly choose a digit
            result.append(random.choice(string.digits))
        elif char.isupper():
            # Randomly choose an uppercase letter
            result.append(random.choice(string.ascii_uppercase))
        elif char.islower():
            # Randomly choose a lowercase letter
            result.append(random.choice(string.ascii_lowercase))
        elif char == '-':
            # Preserve dash
            result.append('-')
        else:
            # Preserve spaces or other non-alphanumeric characters
            result.append(char)

    return ''.join(result)

# todo remove def generate_random_number(min_val=1, max_val=9):
#     """Generate a random number between min_val and max_val (inclusive)."""
#     return random.randint(min_val, max_val)

def generate_random_decimal(min_val=1.00, max_val=99.99, decimals=2):
    """Generate a random number with a specified number of decimal places."""
    return round(random.uniform(min_val, max_val), decimals)

def generate_similar_number(number_str):
    # Detect if the decimal separator is a comma or a period
    if ',' in number_str and '.' in number_str:
        thousand_sep = ','
        decimal_sep = '.'
    elif ',' in number_str and ' ' in number_str:
        decimal_sep = ','
        thousand_sep = ' '
    elif '.' in number_str and ' ' in number_str:
        decimal_sep = '.'
        thousand_sep = ' '
    elif ',' in number_str :
        decimal_sep = ','
        thousand_sep = None
    else:
        decimal_sep = '.'
        thousand_sep = None  # No thousand separator

    # Split number into integer and decimal parts
    if decimal_sep in number_str:
        integer_part, decimal_part = number_str.split(decimal_sep)
    else:
        integer_part = number_str
        decimal_part = ""

    # Remove thousand separators from integer part if present
    if thousand_sep:
        integer_part_clean = integer_part.replace(thousand_sep, '')
    else:
        integer_part_clean = integer_part

    # Convert the cleaned integer part to a float
    base_number = float(integer_part_clean.replace('.', '').replace(',', ''))

    # Generate a random percentage between 1 and 150
    percentage = random.uniform(-0.50, 1.5)
    # Calculate the offset
    offset = base_number * percentage
    # Add the offset to the original number
    new_number = base_number + offset

    return round(new_number, len(decimal_part))

def format_number_like(number, format_str, name=''):

    # Detect decimal separator and split integer and decimal format parts
    if '.' in format_str:
        integer_format, decimal_format = format_str.split('.')
        decimal_separator = '.'
    elif ',' in format_str and format_str.count(',') == 1:
        integer_format, decimal_format = format_str.split(',')
        decimal_separator = ','
    else:
        integer_format = format_str
        decimal_format = ""
        decimal_separator = ""

    # Check for thousand separator in the integer format
    if ' ' in integer_format:
        use_thousand_separator = True
        thousand_separator = ' '
        integer_format = integer_format.replace(' ', '')
    elif ',' in integer_format and len(integer_format) > 1:
        use_thousand_separator = True
        thousand_separator = ','
        integer_format = integer_format.replace(',', '')
    elif '.' in integer_format and len(integer_format) > 1:
        use_thousand_separator = True
        thousand_separator = '.'
        integer_format = integer_format.replace('.', '')
    else:
        use_thousand_separator = False
        thousand_separator = ""

    # Get the number of digits required for the decimal part
    num_decimal_digits = len(decimal_format)

    # Format the number to the required decimal places
    formatted_number = f"{number:.{num_decimal_digits}f}"

    # Split the formatted number into integer and decimal parts
    if '.' in formatted_number:
        integer_part, decimal_part = formatted_number.split('.')
    else:
        integer_part = formatted_number
        decimal_part = ""

    # Adjust the decimal part to the required number of digits
    decimal_part = decimal_part.ljust(num_decimal_digits, '0')

    # Add thousand separators to the integer part if required
    if use_thousand_separator and len(integer_part) > 3:
        integer_part = f"{int(integer_part):,}".replace(",", thousand_separator)

    # Combine the integer and decimal parts with the correct separator
    if decimal_separator:
        result = f"{integer_part}{decimal_separator}{decimal_part}"
    else:
        result = integer_part

    return result

# def generate_fraction():
#     """Generate a random fraction in the format whole_number-numerator/denominator."""
#     whole_number = random.randint(0, 9)
#     numerator = random.randint(0, 31)
#     denominator = 32  # Keeping the denominator fixed at 32 for consistency
#     if numerator == 0:
#         return f"{whole_number}"  # Return just the whole number if numerator is 0
#     return f"{whole_number}-{numerator}/{denominator}"

def substitute_text(data):
    """
    Substitute the text in the input string based on the type of data,
    keeping the rest of the text unchanged and removing curly braces and type.

    Args:
    - data (str): Input string that may contain multiple types and values.

    Returns:
    - str: Modified string with substituted values and curly braces removed.
    """
    # Define regex patterns to find the types and values
    customer_pattern = r'\{"CUSTOMER":\s*([^}]+)\}'
    invoice_number_pattern = r'\{"INVOICE_NUMBER":(\d+)\}'
    po_pattern = r'\{"PO":(\d+)\}'
    qty2_pattern = r'\{"QTY2":(\d+(\.\d+)?)\}'
    text_pattern = r'\{"ITEM_NAME":(.*?)\}'
    n1_pattern = r'\{"N1":(\d)\}'
    unit_price_pattern = r'\{"UNIT_PRICE":(-?\d{1,3}(?:[ ,]\d{3})*(?:[.,]\d+)?)\}'
    itemcode_pattern = r'\{"ITEMCODE":(.*?)\}'
    line_total_pattern = r'\{"LINE_TOTAL":(-?\d{1,3}(?:[ ,]\d{3})*(?:[.,]\d+)?)\}'
    tps_pattern = r'\{"GST":(-?\d{1,3}(?:[ ,]\d{3})*(?:[.,]\d+)?)\}'
    gst_id_pattern = r'\{"GST_ID":\s*([^}]+)\}'
    qst_pattern = r'\{"QST":(-?\d{1,3}(?:[ ,]\d{3})*(?:[.,]\d+)?)\}'
    qst_id_pattern = r'\{"QST_ID":\s*([^}]+)\}'
    subtotal_pattern = r'\{"SUB_TOTAL":(-?\d{1,3}(?:[ ,]\d{3})*(?:[.,]\d+)?)\}'
    fraction_pattern = r'\{"FRACTION":(\d+)-(\d+)/(\d+)\}'
    total_pattern = r'\{"TOTAL":(-?\d{1,3}(?:[ ,]\d{3})*(?:[.,]\d+)?)\}'
    currency_pattern = r'\{"CURRENCY":\s*([^}]+)\}'
    current_invoice_index = 0

    # Store previous QTY2, PRICE4 values and total line total
    total_line_total = 0.0
    labels = {}
    current_line = None
    invoice_meta_data = None
    sub_total = 0
    tvq_value = 0
    tps_value = 0
    current_price = None
    current_quantity = None
    temporization_mode = False
    invoice_complete_text_temporary = ''

    def get_line_number_from_position(text, position):
      # Split the text into lines
      lines = text.splitlines()

      # Accumulate character positions to track the start and end of each line
      char_count = 0

      # Iterate through the lines and find where the position falls
      for line_number, line in enumerate(lines, start=1):
          # Calculate start and end position of the current line
          line_length = len(line) + 1  # +1 for the newline character

          # Check if the position is within this line
          if char_count <= position < char_count + line_length:
              return line_number

          # Update the character count for the next line
          char_count += line_length

      return None  # If position is out of range


    def add_invoice_complete_text(text):
      nonlocal invoice_meta_data
      nonlocal invoice_complete_text_temporary

      if not invoice_meta_data:
        add_new_invoice()

      # Retrieve the metadata of the most recent invoice
      current_invoice = invoice_meta_data[-1]

      if current_invoice['invoice_complete_text'] == None:
        current_invoice['invoice_complete_text'] = ['']

      # Append the provided text with a newline to the last string in 'invoice_complete_text'
      current_invoice['invoice_complete_text'][-1] += text + '\n'


    def add_invoice_new_complete_text(text):
      nonlocal invoice_meta_data

      # Retrieve the metadata of the most recent invoice
      current_invoice = invoice_meta_data[-1]

      # Append the provided text with a newline to the last string in 'invoice_complete_text'
      current_invoice['invoice_complete_text'].append(text)

    def add_invoice_complete_text_temporary(text):
      nonlocal invoice_complete_text_temporary
      invoice_complete_text_temporary += text + '\n'

    def add_new_invoice(text = None):
      nonlocal invoice_meta_data
      if not invoice_meta_data:
        invoice_meta_data = []
      invoice_meta_data.append({'lines': [],'invoice_complete_text': [text] if text else None})

    def get_current_line():
      nonlocal invoice_meta_data
      if invoice_meta_data:
          current_invoice = invoice_meta_data[-1]
          if current_invoice and 'lines' in current_invoice and current_invoice['lines']:
              return current_invoice['lines'][-1]

      return None


    def add_label(row, label):
        nonlocal temporization_mode
        nonlocal invoice_complete_text_temporary
        nonlocal invoice_meta_data

        # Extract key and value from label
        key, value = next(iter(label.items()))

        if not invoice_meta_data:
          add_new_invoice()

        current_invoice = invoice_meta_data[-1]

        if not current_invoice['lines']:
          current_invoice['lines'].append({})

        current_line = current_invoice['lines'][-1]


        if key in ['item_code', 'item_name', 'quantity', 'quantity_str', 'unit_price','unit_price_str', 'line_total','line_total_str']:
            if key in current_line:
                # Start a new line if key already exists
                current_line = {key: value}
                current_invoice['lines'].append(current_line)
            else:
                current_line[key] = value  # Add to the current line
        else:
            # New invoice
            if key in current_invoice and current_invoice[key] == value:
                add_new_invoice(invoice_complete_text_temporary)
                add_label(row, label)

                # Reset split mode
                temporization_mode = False
                invoice_complete_text_temporary = ''

            # Existing invoice
            else:
                current_invoice[key] = value
                if temporization_mode:
                  add_invoice_new_complete_text(invoice_complete_text_temporary)

                  # Reset split mode
                  temporization_mode = False
                  invoice_complete_text_temporary = ''


    def replace_item_name(match):
        item_name = generate_similar_string(match.group(1))
        line_index = get_line_number_from_position(result, match.start())
        add_label(line_index, {"item_name":item_name})

        return item_name

    # def replace_n1(match):
    #     return str(generate_random_number())
    def replace_n1(match):
        return generate_similar_number(match.group(1))

    # todo remove def get_number(match, min_val=1, max_val=9):
    #     return str(generate_random_number(min_val=min_val, max_val=max_val))

    def replace_qty2(match):
        nonlocal current_quantity
        current_quantity = generate_similar_number(match.group(1))

        line_index = get_line_number_from_position(result, match.start())
        add_label(line_index, {"quantity":current_quantity})
        add_label(line_index, {"quantity_str":format_number_like(current_quantity,match.group(1),"quantity") })
        return str(current_quantity)

    def replace_price4(match):

        # Retrieve the current line
        current_line = get_current_line()

        # Check if unit price is provided
        current_price = current_line['unit_price'] if 'unit_price' in current_line else None

        line_index = get_line_number_from_position(result, match.start())

        if current_price is None:
          current_price = generate_similar_number(match.group(1))
          add_label(line_index, {"unit_price":current_price})

        current_price_str = format_number_like(current_price, match.group(1),"current_price")
        add_label(line_index, {"unit_price_str":current_price_str})

        return current_price_str

    def replace_itemcode(match):
        value = match.group(1)

        new_item_code = generate_similar_string(value)

        line_index = get_line_number_from_position(result, match.start())
        add_label(line_index, {"item_code": new_item_code})

        return new_item_code

    def replace_line_total(match, index):
        nonlocal total_line_total
        nonlocal current_quantity

        line_total = 0

        # Retrieve the current line
        current_line = get_current_line()

        # Check if Quantity is provided
        current_quantity = current_line['quantity'] if current_line is not None and 'quantity' in current_line else None

        # Check if unit price is provided
        unit_price = current_line['unit_price'] if current_line is not None and 'unit_price' in current_line else None

        if current_quantity and unit_price:
            line_total = current_quantity * unit_price

        line_index = get_line_number_from_position(result, match.start())

        # Price is not provided
        if current_quantity and unit_price is None:

            # Create new line total
            line_total = generate_similar_number(match.group(1))

            # Set previous price
            unit_price = line_total / current_quantity
            add_label(line_index, {"unit_price": unit_price})

        total_line_total += line_total
        line_total_str = format_number_like(line_total, match.group(1),"line_total")
        add_label(line_index, {"line_total": line_total})
        add_label(line_index, {"line_total_str": line_total_str })

        return line_total_str

    def replace_tps(match):
        nonlocal tps_value

        if len(match.group(1)) == 0:
          return ''

        # Calculate TPS as a fixed percentage of the total line total
        tps_rate = 0.05  # Example tax rate of 15%
        tps_value = total_line_total * tps_rate

        line_index = get_line_number_from_position(result, match.start())
        tps_value_str = format_number_like(tps_value, match.group(1),"gst")
        add_label(line_index, {"gst":tps_value})
        add_label(line_index, {"gst_str":tps_value_str})

        return tps_value_str

    def add_gst_id(match):
        line_index = get_line_number_from_position(result, match.start())
        add_label(line_index, {"gst_id":match.group(1)})
        return match.group(1)

    def add_qst_id(match):
        line_index = get_line_number_from_position(result, match.start())
        add_label(line_index, {"qst_id":match.group(1)})
        return match.group(1)

    def replace_qst(match):
        nonlocal tvq_value
        # Calculate TVQ as a fixed percentage of the total line total
        tvq_rate = 0.09975  # Example Quebec TVQ rate (9.975%)
        tvq_value = total_line_total * tvq_rate
        tvq_value_str = format_number_like(tvq_value, match.group(1),"tvq")
        line_index = get_line_number_from_position(result, match.start())
        add_label(line_index, {"qst": tvq_value})
        add_label(line_index, {"qst_str":tvq_value_str})

        return f"{tvq_value:.2f}"

    def replace_subtotal(match):
        line_index = get_line_number_from_position(result, match.start())
        add_label(line_index, {"sub_total": total_line_total})
        sub_total_str = format_number_like(total_line_total, match.group(1),"sub_total")
        add_label(line_index, {"sub_total_str": sub_total_str })

        # Replace SUB_TOTAL with the sum of all line total values
        return sub_total_str

    def replace_total(match):
        nonlocal total_line_total
        nonlocal tps_value
        nonlocal tvq_value

        line_index = get_line_number_from_position(result, match.start())
        total = total_line_total + tvq_value + tps_value
        total_str = format_number_like(total, match.group(1),"total")
        add_label(line_index, {"total": total})
        add_label(line_index, {"total_str": total_str})

        # Replace SUB_TOTAL with the sum of all line total values
        return total_str

    # def replace_fraction(match):
    #     return generate_fraction()

    def replace_po(match):

        def is_integer(s):
          try:
              int(s)
              return True
          except ValueError:
              return False

        if is_integer(match.group(1)):
          po_number = generate_similar_number(match.group(1))
          po_number_str = format_number_like(po_number, match.group(1),"po_number")
        else:
          po_number = generate_similar_string(match.group(1))
          po_number_str = po_number

        line_index = get_line_number_from_position(result, match.start())
        add_label(line_index, {"po_number": po_number})
        add_label(line_index, {"po_number_str": po_number_str})
        return po_number_str

    def replace_invoice_number(match):
        invoice_number = generate_similar_number(match.group(1))
        invoice_number_str = format_number_like(invoice_number, match.group(1),"invoice_number")
        line_index = get_line_number_from_position(result, match.start())
        add_label(line_index, {"invoice_number": invoice_number})
        add_label(line_index, {"invoice_number_str": invoice_number_str})
        return invoice_number_str

    def replace_customer(match):
        customer_name = match.group(1)
        line_index = get_line_number_from_position(result, match.start())
        add_label(line_index, {"customer": f"{customer_name}"})
        return customer_name

    def add_currency(match):
        line_index = get_line_number_from_position(result, match.start())
        add_label(line_index, {"currency": match.group(1)})
        return match.group(1)

    # Initialize the cumulative index with 1, stored in a mutable list
    index = [0]

    invoice_text = []
    current_price = None
    current_quantity = None
    tps_value = 0
    tvq_value = 0
    read_enable = False

    # Split the text into lines
    for line in data.splitlines():

        if '{"BEGIN"}' in line:
          read_enable = True
          continue

        if '{"END"}' in line:
          read_enable = False
          continue

        if not read_enable:
          continue

        # Replace text patterns and remove curly braces and type
        result = re.sub(text_pattern, lambda m: replace_item_name(m), line)

        # Replace INVOICE_NUMBER patterns and remove curly braces and type
        result = re.sub(invoice_number_pattern, replace_invoice_number, result)

        # Replace N1 patterns and remove curly braces and type
        result = re.sub(n1_pattern, replace_n1, result)

        # Replace QTY2 patterns and remove curly braces and type
        result = re.sub(qty2_pattern, replace_qty2, result)

        # Replace unit price patterns and remove curly braces and type
        result = re.sub(unit_price_pattern, replace_price4, result)

        # Replace ITEMCODE patterns and remove curly braces and type
        result = re.sub(itemcode_pattern, replace_itemcode, result)

        # Replace LINE_TOTAL patterns and update total line total
        result = re.sub(line_total_pattern, lambda m: replace_line_total(m, index), result)

        # Replace GST patterns with calculated value
        result = re.sub(tps_pattern, replace_tps, result)

        # Replace GST_ID patterns with calculated value
        result = re.sub(gst_id_pattern, add_gst_id, result)

        # Replace QST patterns with calculated value
        result = re.sub(qst_pattern, replace_qst, result)

        # Replace QST_ID patterns with calculated value
        result = re.sub(qst_id_pattern, add_qst_id, result)

        # Replace SUB_TOTAL patterns with the sum of line totals
        result = re.sub(subtotal_pattern, replace_subtotal, result)

        # Replace TOTAL patterns with the sum of line totals
        result = re.sub(total_pattern, replace_total, result)

        # todo remove # Replace FRACTION patterns and remove curly braces and type
        # result = re.sub(fraction_pattern, replace_fraction, result)

        # Replace PO patterns and remove curly braces and type
        result = re.sub(po_pattern, replace_po, result)

        # Replace CURRENCY patterns and remove curly braces and type
        result = re.sub(currency_pattern, add_currency, result)

        result = re.sub(customer_pattern, replace_customer, result)

        if '{"SPLIT"}' in result:
          part1, part2 = result.split('{"SPLIT"}')
          add_invoice_complete_text(part1)
          add_invoice_complete_text_temporary(part2)
        elif temporization_mode:
          add_invoice_complete_text_temporary(result)
        else:
          add_invoice_complete_text(result)

    return invoice_meta_data

def split_text_on_split(text):
    # Split the text by the word {"SPLIT"}
    sections = text.split('{"SPLIT"}')

    # Create an array to store the sections and combined elements
    output_array = []

    # Add each section as an element to the array
    for i, section in enumerate(sections):
        # Split the section into rows (lines)
        section_lines = section.strip().splitlines()

        if i == 0:
          output_array.append('\n'.join(section_lines))
        else:
          prev_section_lines = sections[i - 1].strip().splitlines()

          current_section = '\n'.join(section_lines)
          prev_section_lines = ''#todo '\n'.join(prev_section_lines[-10:])
          combined_lines = current_section + prev_section_lines
          output_array.append(combined_lines)

    return output_array


# Specify the directory where the files are located
directory_path = '/content/'

# Use glob to find all files ending with '_conv.txt' in the directory
file_pattern = os.path.join(directory_path, '*.txt')
files = glob.glob(file_pattern)

# Iterate over each file and read its content
reset_dataset()

for file_path in files:
    print('file:',file_path)
    with open(file_path, 'r') as file:
      invoice_template = file.read()

    for c in range(10):
      invoice_meta_datas = substitute_text(invoice_template)

      for invoice_meta_data in invoice_meta_datas:
        converted = convert_to_structure(invoice_meta_data)

        converted_str = json.dumps(converted)

        choices = ['train', 'validation', 'test']
        probabilities = [0.6, 0.2, 0.2]  # Corresponding probabilities

        # Use random.choices to select based on weighted probabilities
        split = random.choices(choices, probabilities)[0]

        for full_text_split in invoice_meta_data['invoice_complete_text']:
          filename = os.path.basename(file_path)
          add_to_dataset_straigh(full_text_split, converted_str, split, filename)

print(data_all)
print('completed')

file: /content/Facture 75560.txt
{'train': {'text': ['                                                               Date :                 2024-07-04\n                                                            Termes  :              Net 30 jours/daysCHEM ACTION INC.\n                                                             # Connaissement  :     1ZA1A3922099024053 4559, MÉTROPOLITAIN EST\n ST-LÉONARD, QUÉBEC, CANADA\nH1R 1Z4          Tél. : (514) 593-1515                            Votre commande :      {"PO":540091256:1555}\n               Fax :(514) 593-1313                            Transporteur :       UPS (COLLECT)\n                                                             # Compte :            # 22R6V9\n     Client : DIS1280                                                               Expédier à :  MET2151       GROUPE LD INC.\n                                                                   ALUMINERIE DE BECANCOUR INC.\n                                              

#Save into Hub dataset or stay local

In [ ]:
def convert_dataset_dict_to_dataset(dataset_dict):
  # Initialize a new dictionary to hold the data in the required format
  converted_dict = {}

  # Iterate through the DatasetDict and populate the new dictionary
  for split in dataset_dict:
    converted_dict[split] = {
        'text': dataset_dict[split]['text'],  # Extract 'text' column
        'json': dataset_dict[split]['json']   # Extract 'json' column
    }

  return converted_dict

def merge_dataset_dict(json1,json2):
  # Initialize a new dictionary to hold the merged result
  merged_json = {}

  for key in json1:
    if key in json2:
        # If the key exists in both JSONs, concatenate the lists
        merged_json[key] = {
            'text': json1[key]['text'] + json2[key]['text'],
            'json': json1[key]['json'] + json2[key]['json']
        }
    else:
        # If the key only exists in json1
        merged_json[key] = json1[key]
  # Check for keys that are in json2 but not in json1
  for key in json2:
      if key not in merged_json:
          # If the key only exists in json2
          merged_json[key] = json2[key]

  return merged_json

#update dataset
# from datasets import load_dataset
# dataset_name_update = "pajacques/yes"
# dataset_dict = load_dataset(dataset_name_update)
# old_data_all = convert_dataset_dict_to_dataset(dataset_dict)

# data_all = merge_dataset_dict(old_data_all, data_all)

#push dataset
push_dataset_to_hf(data_all)

Uploading the dataset shards:   0%|          | 0/1 [00:00<?, ?it/s]

Creating parquet from Arrow format:   0%|          | 0/1 [00:00<?, ?ba/s]

Uploading the dataset shards:   0%|          | 0/1 [00:00<?, ?it/s]

Creating parquet from Arrow format:   0%|          | 0/1 [00:00<?, ?ba/s]

Uploading the dataset shards:   0%|          | 0/1 [00:00<?, ?it/s]

Creating parquet from Arrow format:   0%|          | 0/1 [00:00<?, ?ba/s]

README.md:   0%|          | 0.00/549 [00:00<?, ?B/s]

#COunt number

In [ ]:
from transformers import AutoTokenizer

# Load the tokenizer
model_id = "your-model-id"
tokenizer = AutoTokenizer.from_pretrained(model_id)

# Adjust tokenizer settings if needed
tokenizer.padding_side = 'right'
tokenizer.pad_token = tokenizer.eos_token

# Input string
text = "This is an example string to tokenize."

# Tokenize and count tokens
tokens = tokenizer.encode(text, truncation=False, add_special_tokens=True)  # Add special tokens if needed
num_tokens = len(tokens)

print(f"Number of tokens: {num_tokens}")


# Step 3 - Fine tuning using llama3


## Install dependencies

In [ ]:
# install dependencies
!pip install --quiet --update transformers

# we use the latest version of transformers, peft, and accelerate
!pip install -q accelerate peft transformers

# install bitsandbytes for quantization
!pip install -q bitsandbytes
#!pip install -U bitsandbytes

# install trl for the SFT library
!pip install -q trl

# we need to install datasets for our training dataset
!pip install -q datasets

# we need huggingface hub to get access to the llama 3 model
!pip install huggingface_hub


Usage:   
  pip3 install [options] <requirement specifier> [package-index-options] ...
  pip3 install [options] -r <requirements file> [package-index-options] ...
  pip3 install [options] [-e] <vcs project url> ...
  pip3 install [options] [-e] <local project path> ...
  pip3 install [options] <archive url/path> ...

no such option: --update
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 122.4/122.4 MB 17.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 310.9/310.9 kB 7.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 480.6/480.6 kB 19.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 8.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.3/179.3 kB 14.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 9.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 14.1 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that

## Login to huggingface

In [ ]:
from huggingface_hub import login
from google.colab import userdata

token = userdata.get('HF_TOKEN')

# Log in to Hugging Face
login(token, add_to_git_credential=True)

## Initialize variables
The following configures our settings for finetuning our model

In [ ]:
# we will use llama-3 instruct as our base model
model_id = "meta-llama/Meta-Llama-3-8b-Instruct"
#model_id = "meta-llama/Llama-3.1-70B-Instruct"

# The instruction dataset to use
dataset_name = "pajacques/yes"# "chrishayuk/calvin_scale_classifications_llama3"

# Fine-tuned model name
new_model = "llama-3-8b-calvinscale_yes"
#new_model = "meta-llama/Llama-3.1-70B-Instruct_yes"

# Output directory where the model predictions and checkpoints will be stored
output_dir = "./results"

# Number of training epochs
num_train_epochs = 1

#Count the token

In [ ]:
from transformers import AutoTokenizer

# Load the tokenizer
model_id = model_id
tokenizer = AutoTokenizer.from_pretrained(model_id)

# Adjust tokenizer settings if needed
tokenizer.padding_side = 'right'
tokenizer.pad_token = tokenizer.eos_token

# Input string
text = '''{"invoice_number": "4757", "po_number": "293466", "custmer_no": "M09941", "gst_tax_id": "882118318RC0001", "qst_tax_id": "1088825418TQ0001", "lines": [{"item_code": "BC51-85623", "item_name": "Bjojcmfc uudpts 1\"ut x 1\"t (sbj qwiegf 0-78/04\"us x 0-7/8\"q)", "quantity": "17.25", "unit_price": "1.3299", "line_total": "22.94"}, {"item_code": "DM35-22095", "item_name": "Hxcbqaea uwmpyu 1\"yg x 4\"v (zvi nsnmcm 2-61/26\"pa x 1-4/6\"z)", "quantity": "8.04", "unit_price": "1.9629", "line_total": "15.78"}, {"item_code": "LF25-31334", "item_name": "Rpwywgfu mbpfzc 9\"ct x 4\"n (drg ayxele 3-48/40\"ys x 9-3/7\"y)", "quantity": "11.33", "unit_price": "1.4078", "line_total": "15.95"}, {"item_code": "NU53-20928", "item_name": "Ktiwcmru imjpdr 2\"wp x 80\"h (fhe wcckwa 8-89/81\"mt x 64-5/9\"x)", "quantity": "11.12", "unit_price": "6.0580", "line_total": "67.36"}, {"item_code": "DD-7ESR46152L", "item_name": "Mahfb s hqzvof ebvftk, 6 otktziu (26\"hv x 72\"kn x 29-7/8\"n)", "quantity": "1.63", "unit_price": "2 179.1024", "line_total": "3 551.94"}, {"item_code": "L63CE-A026611", "item_name": "Zhvzwlyxfkffvvoux pb lyqksn (E) 37\"av x 56\"zs x 5\"z (96 jmyw)", "quantity": "2.38", "unit_price": "29.7084", "line_total": "70.71"}, {"item_code": "M69ZL-T185102", "item_name": "Eysoaoudjmrpqiljn fm lwosvb (L) 20\"rh x 21\"rk x 9\"w (37 qxcp)", "quantity": "2.15", "unit_price": "120.1658", "line_total": "258.36"}, {"item_code": "V83FM-U138703", "item_name": "Oruobeyrgjxpezbfs yu vvqfdy (F) 29\"qk x 19\"yy x 0\"m (07 lubf)", "quantity": "1.15", "unit_price": "121.3723", "line_total": "139.58"}, {"item_code": "O96CR-O655180", "item_name": "Poxedmbrenxlrpjmc et hinwna (W) 03\"ld x 24\"sa x 5\"o (31 rpla)", "quantity": "0.65", "unit_price": "111.0624", "line_total": "72.19"}, {"item_code": "Z75NP-M064172", "item_name": "Zucxfbcecxdyeldpe yo mjxxqm (C) 87\"zt x 68\"uh x 3\"c (62 xbfu)", "quantity": "0.60", "unit_price": "220.1588", "line_total": "132.10"}, {"item_code": "ZB30-4044F", "item_name": "Jzjvut is tvsv nuhcnzw 24\"ua x 24\"si x 2 \u00be\"u (zuvcnxb)", "quantity": "2.29", "unit_price": "459.5270", "line_total": "1 052.32"}]}'''

# Tokenize and count tokens
tokens = tokenizer.encode(text, truncation=False, add_special_tokens=True)  # Add special tokens if needed
num_tokens = len(tokens)

print(f"Number of tokens: {num_tokens}")


tokenizer_config.json:   0%|          | 0.00/51.0k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/9.09M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/73.0 [00:00<?, ?B/s]

Number of tokens: 960


## Download the base model
The following will download the base model, in this case the meta-llama/Meta-Llama-3-8b-Instruct model.

In [ ]:
# import torch
# from transformers import (
#     AutoTokenizer,
#     AutoModelForCausalLM,
#     BitsAndBytesConfig,
#     pipeline,
#     logging,
# )

# # load the quantized settings, we're doing 4 bit quantization
# bnb_config = BitsAndBytesConfig(
#     load_in_4bit=True,
#     bnb_4bit_quant_type="nf4",
#     bnb_4bit_compute_dtype=torch.bfloat16,
#     bnb_4bit_use_double_quant=False,
# )

# # Load base model
# model = AutoModelForCausalLM.from_pretrained(
#     model_id,
#     quantization_config=bnb_config,
#     # use the gpu
#     device_map={"": 0}
# )

# # don't use the cache
# model.config.use_cache = False

config.json:   0%|          | 0.00/654 [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/23.9k [00:00<?, ?B/s]

model-00001-of-00004.safetensors:   0%|          | 0.00/4.98G [00:00<?, ?B/s]

model-00002-of-00004.safetensors:   0%|          | 0.00/5.00G [00:00<?, ?B/s]

model-00003-of-00004.safetensors:   0%|          | 0.00/4.92G [00:00<?, ?B/s]

model-00004-of-00004.safetensors:   0%|          | 0.00/1.17G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/187 [00:00<?, ?B/s]

In [ ]:
# # Initialize the tokenizer with specific adjustments
# tokenizer = AutoTokenizer.from_pretrained(model_id)
# tokenizer.padding_side = 'right'
# tokenizer.pad_token = tokenizer.eos_token
# #tokenizer.chat_template = get_llama3_chat_template()

tokenizer_config.json:   0%|          | 0.00/51.0k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/9.09M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/73.0 [00:00<?, ?B/s]

#Extracting Text from PDF and Formatting with Prompt Templates

In [ ]:
# !pip install --quiet pymupdf

In [ ]:
# import sys, pathlib, fitz
# def getText(fname):
#     all_text = ""  # Initialize an empty string to store the concatenated text
#     with fitz.open(fname) as pages:
#         for page in pages:
#             words = page.get_text("text", sort=True)  # Get the text of the current page
#             all_text += words  # Append the text of the page to the 'all_text' variable
#     return all_text

# def get_prompt(fname):
#   with open("/content/prompt_template.txt", "r") as file: #file: "C:\Code\Peso\exemple invoice\prompt_template.txt"
#     template = file.read()
#   template = template.replace("#####", getText(fname))
#   return template

# def get_prompt_from_text(text):
#   with open("/content/prompt_template.txt", "r") as file: #file: "C:\Code\Peso\exemple invoice\prompt_template.txt"
#     template = file.read()
#   template = template.replace("#####", text)
#   return template

### Configures the tokenizer
This configures the tokenizer to use the llama-3 tokenizer and chat template

## Run the Model (prior to fine tuning)

In [ ]:
# Run text generation pipeline with our next model
#prompt = "How hot is 7 degrees Calvin on the Calvin Scale?"
#prompt = "<|begin_of_text|><|start_header_id|>system<|end_header_id|>{'prompt': 'You are a helpful assistant'}<|eot_id|><|start_header_id|>user<|end_header_id|>How hot is 122 degrees Calvin on the Calvin Scale?<|eot_id|><|start_header_id|>assistant<|end_header_id|>"
#prompt = "<|begin_of_text|><|start_header_id|>system<|end_header_id|>{'prompt': 'You are a helpful assistant'}<|eot_id|><|start_header_id|>user<|end_header_id|>Write a Poem about the Calvin Scale<|eot_id|><|start_header_id|>assistant<|end_header_id|>"
#prompt = "Who is Ada Lovelace?"
#prompt = "<|begin_of_text|><|start_header_id|>system<|end_header_id|>{'prompt': 'You are a helpful assistant'}<|eot_id|><|start_header_id|>user<|end_header_id|>Who is Ada Lovelace?<|eot_id|><|start_header_id|>assistant<|end_header_id|>"
#prompt = "What would you classify the typical weather in Arizona in the summer as according to the Calvin Scale?"
#prompt = "How hot is 9 degrees Calvin on the Calvin Scale?"
#prompt = "Write a poem about the Calvin Scale in limerick style"
#prompt = "What is the capitalof canada"

prompt = get_prompt('/content/PSAProjInvoice.Report (6).pdf')

# execute the query
pipe = pipeline(task="text-generation", model=model, tokenizer=tokenizer, max_length=4000)
result = pipe(f"{prompt}")

# show the result
#USE: generation_output[0][input_ids.shape[-1]:]
#inputs = self.tokenizer(query, return_tensors='pt').to(self.model.device)
#outputs = self.model.generate(
#    input_ids=inputs['input_ids'],
#    attention_mask=inputs['attention_mask'],
#    generation_config=gen_config
#)
#output_str = self.tokenizer.decode(outputs[0], skip_special_tokens=True)
print(result[0]['generated_text'])

# Train the Model

## Load Dataset


In [ ]:
# from datasets import load_dataset

# # Load the dataset
# dataset = load_dataset(dataset_name, split="train")

README.md:   0%|          | 0.00/603 [00:00<?, ?B/s]

train-00000-of-00001.parquet:   0%|          | 0.00/5.20M [00:00<?, ?B/s]

validation-00000-of-00001.parquet:   0%|          | 0.00/1.70M [00:00<?, ?B/s]

test-00000-of-00001.parquet:   0%|          | 0.00/1.73M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/4834 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/1570 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/1596 [00:00<?, ? examples/s]

### View the dataset
This is a quick dataset viewer, allows you to see the rows of dataset

In [ ]:
import pandas as pd

# Convert to pandas DataFrame
df = pd.DataFrame(dataset)

# Display the first few rows of the DataFrame to understand its structure
print(df.head())

# Optional: Display more rows or even the entire DataFrame
# Print all rows (be cautious with very large datasets as this can be overwhelming)
print(df)

# Print a specific number of rows, for example, the first 20 rows
print(df.head(20))

## Fine Tune the model #1
We can now kick off the training loop

In [ ]:
# import torch
# from transformers import (
#     AutoModelForCausalLM,
#     AutoTokenizer,
#     BitsAndBytesConfig,
#     HfArgumentParser,
#     TrainingArguments,
#     pipeline,
#     logging,
# )
# from peft import LoraConfig, PeftModel
# from trl import SFTTrainer

# # Load LoRA configuration
# peft_config = LoraConfig(
#     lora_alpha=16,
#     lora_dropout=0.1,
#     r=64,
#     bias="none",
#     task_type="CAUSAL_LM",
# )

# # Set training parameters
# training_arguments = TrainingArguments(
#     output_dir=output_dir,
#     num_train_epochs=num_train_epochs,      # uses the number of epochs earlier
#     per_device_train_batch_size=2,          # 2 seems reasonable (made smaller due to CUDA memory issues)
#     gradient_accumulation_steps=2,          # 2 is fine, as we're a small batch
#     optim="paged_adamw_32bit",              # default optimizer
#     save_steps=0,                           # we're not gonna save
#     logging_steps=10,                       # same value as used by Meta
#     learning_rate=2e-4,                     # standard learning rate
#     weight_decay=0.001,                     # standard weight decay 0.001
#     fp16=False,                             # set to true for A100
#     bf16=True,                             # set to true for A100
#     max_grad_norm=0.3,                      # standard setting
#     max_steps=240,                           # needs to be -1, otherwise overrides epochs
#     warmup_ratio=0.03,                      # standard warmup ratio
#     group_by_length=True,                   # speeds up the training
#     lr_scheduler_type="cosine",           # constant seems better than cosine
#     report_to="tensorboard",
#     evaluation_strategy="steps",            # evaluate at each `eval_steps`
#     eval_steps=10,                          # evaluate every 10 steps
#     save_total_limit=2,                     # keep only the 2 most recent models
#     load_best_model_at_end=True,            # load the best model at the end of training
#     metric_for_best_model="loss"            # use the loss metric to choose the best
# )

# # Set supervised fine-tuning parameters
# trainer = SFTTrainer(
#     model=model,
#     train_dataset=dataset,
#     peft_config=peft_config,                # use our lora peft config
#     dataset_text_field="text",
#     max_seq_length=None,                    # no max sequence length
#     tokenizer=tokenizer,                    # use the llama tokenizer
#     args=training_arguments,                # use the training arguments
#     packing=False,                          # don't need packing
# )

# # Train model
# trainer.train()

# # Save trained model
# trainer.model.save_pretrained(new_model)

#Fine Tune model #2

#Logging to hufgging face

In [1]:
from huggingface_hub import login
from google.colab import userdata

token = userdata.get('HF_TOKEN')

# Log in to Hugging Face
login(token, add_to_git_credential=True)

#Instal dependencies

In [2]:
# install dependencies
!pip install --quiet --update transformers

# we use the latest version of transformers, peft, and accelerate
!pip install -q accelerate peft transformers

# install bitsandbytes for quantization
!pip install -q bitsandbytes
#!pip install -U bitsandbytes

# install trl for the SFT library
!pip install -q trl

# we need to install datasets for our training dataset
!pip install -q datasets

# we need huggingface hub to get access to the llama 3 model
!pip install huggingface_hub


Usage:   
  pip3 install [options] <requirement specifier> [package-index-options] ...
  pip3 install [options] -r <requirements file> [package-index-options] ...
  pip3 install [options] [-e] <vcs project url> ...
  pip3 install [options] [-e] <local project path> ...
  pip3 install [options] <archive url/path> ...

no such option: --update
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 122.4/122.4 MB 13.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 310.9/310.9 kB 8.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 480.6/480.6 kB 26.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 10.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.3/179.3 kB 18.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 13.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 18.7 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages th

#Initilze variables

In [3]:
# we will use llama-3 instruct as our base model
#model_id = "meta-llama/Meta-Llama-3-8b-Instruct"
#model_id = "meta-llama/Llama-3.1-70B-Instruct"
#model_id = "bert-base-uncased"
#model_id = "google/bigbird-roberta-base"
#model_id = "allenai/longformer-base-4096"
model_id = "meta-llama/Llama-3.1-8B-Instruct"

# The instruction dataset to use
dataset_name = "pajacques/yes"# "chrishayuk/calvin_scale_classifications_llama3"

# Fine-tuned model name
new_model = "llama-3-8b-calvinscale_yes"
#new_model = "meta-llama/Llama-3.1-70B-Instruct_yes"

# Output directory where the model predictions and checkpoints will be stored
output_dir = "./results"

# Number of training epochs
num_train_epochs = 1

In [4]:
def calculate_max_length(dataset, input_column="text", tokenizer=None):
    max_len = 0
    # Iterate over the dataset to calculate the token lengths
    for example in dataset[input_column]:
        tokenized_example = tokenizer(example)
        max_len = max(max_len, len(tokenized_example['input_ids']))
    return max_len

In [5]:
from transformers import AutoModelForTokenClassification
from datasets import load_dataset
from transformers import EarlyStoppingCallback
import torch
from transformers import (
    AutoModelForCausalLM,
    AutoTokenizer,
    BitsAndBytesConfig,
    HfArgumentParser,
    TrainingArguments,
    pipeline,
    logging,
)

from datasets import Dataset
from peft import LoraConfig, PeftModel
from trl import SFTTrainer


# Load tokenizer
tokenizer = AutoTokenizer.from_pretrained(model_id)  # Replace with your model name
tokenizer.padding_side = 'right'
tokenizer.pad_token = tokenizer.eos_token

# Ensure padding token is set
if tokenizer.pad_token is None:
    tokenizer.pad_token = tokenizer.eos_token if tokenizer.eos_token else tokenizer.sep_token


# Preprocess the dataset
# def preprocess_function(examples):
#     # Define the columns to use
#     input_column = "text"   # Column name for the input
#     target_column = "json"  # Column name for the target

#     inputs = examples[input_column]  # Extract the input column
#     targets = examples[target_column]  # Extract the target column
#     model_inputs = tokenizer(inputs, max_length=1000, truncation=True, padding="max_length", return_tensors="pt")

#     # Tokenize the target and add it as labels
#     labels = tokenizer(targets, max_length=1000, truncation=True, padding="max_length")
#     model_inputs["labels"] = labels["input_ids"]
#     return model_inputs

def tokenize_dataset_with_json(dataset, tokenizer, max_length=1000):
    # Initialize a dictionary to hold lists for each key
    data_dict = {
        'input_ids': [],
        'attention_mask': [],
        'labels': []
    }

    # Iterate through the dataset and tokenize the input and labels
    for entry in dataset:
        input_column = "text"   # Column name for the input
        target_column = "json"  # Column name for the target

        # Extract raw text and JSON data
        raw_text = entry[input_column]
        json_data = entry[target_column]  # Structured JSON data (dictionary or list)

        # Tokenize the raw text using the tokenizer
        tokenized_text = tokenizer(
            raw_text,
            truncation=True,
            padding='max_length',
            max_length=max_length,
            return_tensors="pt"
        )

        # Convert the tokenized input to a list of input_ids
        input_ids = tokenized_text["input_ids"][0].tolist()

        # Handle JSON data
        if isinstance(json_data, dict):
            json_str = str(json_data)  # Convert the dictionary to a string
        elif isinstance(json_data, list):
            json_str = " ".join([str(item) for item in json_data])  # Convert list items to a string
        else:
            json_str = str(json_data)  # Fallback: convert to string if it's neither a list nor dict

        # Tokenize the JSON data (labels)
        tokenized_json = tokenizer(
            json_str,
            truncation=True,
            padding='max_length',
            max_length=max_length,
            return_tensors="pt"
        )

        # Convert the tokenized JSON data to a list of token IDs (labels)
        label_ids = tokenized_json["input_ids"][0].tolist()

        # Append the tokenized data to the respective lists in data_dict
        data_dict['input_ids'].append(input_ids)
        data_dict['attention_mask'].append(tokenized_text['attention_mask'][0].tolist())
        data_dict['labels'].append(label_ids)

    # Create the Dataset
    return Dataset.from_dict(data_dict)


# Load the dataset
print('Load dataset...')
train_dataset = load_dataset(dataset_name, split='train')
test_dataset = load_dataset(dataset_name, split='test')
eval_dataset = load_dataset(dataset_name, split='validation')

# Calculate the max length for the inputs
max_length = calculate_max_length(train_dataset, input_column="text", tokenizer=tokenizer)
print(f"Max length for inputs: {max_length}")

# Tokenize the dataset
print('Tokenize train...')
train_dataset_tokenized = tokenize_dataset_with_json(train_dataset,tokenizer)
#print('Tokenize test...')
#test_dataset_tokenized = tokenize_dataset_with_json(test_dataset,tokenizer)
print('Tokenize eval...')
eval_dataset_tokenized = tokenize_dataset_with_json(eval_dataset,tokenizer)

# Load LoRA configuration
peft_config = LoraConfig(
    task_type="CAUSAL_LM",    # for general chat, TOKEN_CLS
    lora_alpha=16,  # Scaling factor for the low-rank matrices example:16
    lora_dropout=0.1, # Dropout applied to the low-rank matrices
    r=64, # Rank of the low-rank matrices
    bias="none",
    #target_modules=["query", "value"],  # Fine-tune attention layers
)

# Set training parameters
print('Set training parameters...')
training_arguments = TrainingArguments(
    output_dir=output_dir,  # Directory to save model checkpoints
    num_train_epochs=5,   # Set the number of epochs as max_steps=-1, num_train_epochs
    per_device_train_batch_size=2,  # Set batch size
    gradient_accumulation_steps=2,  # Gradient accumulation steps
    optim="paged_adamw_32bit",  # Optimizer
    save_steps=10,  # Save model every 10 steps
    logging_steps=5,  # Log every 10 steps
    learning_rate=2e-4,  # Learning rate
    weight_decay=0.001,  # Weight decay
    fp16=False,  # Set to True if using FP16
    bf16=True,  # Set to True for A100 GPUs
    max_grad_norm=0.3,  # Max gradient norm
    max_steps=-1, # Set to -1 to disable max steps
    warmup_ratio=0.03,  # Warmup ratio
    group_by_length=True,         # Group sequences by length
    lr_scheduler_type="cosine",   # Learning rate scheduler
    report_to="tensorboard",      # Report to TensorBoard
    evaluation_strategy="steps",  # Evaluate every few steps
    eval_steps=10,                # Evaluation steps
    save_total_limit=2,           # Limit the number of saved models
    load_best_model_at_end=True,  # Load the best model at the end of training
    metric_for_best_model="loss",  # Use [loss|accuracy] to choose the best model
)


# load the quantized settings, we're doing 4 bit quantization
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.bfloat16,
    bnb_4bit_use_double_quant=False,
)

# Define EarlyStoppingCallback with patience of 3 steps
early_stopping_callback =  EarlyStoppingCallback(
    early_stopping_patience=2,    # Stop after 2 evaluation steps if no improvement
    early_stopping_threshold=0.01 # Only consider stopping if improvement is less than this threshold
)

# Set supervised fine-tuning parameters,AutoModelForCausalLM, AutoModelForTokenClassification
trainer = SFTTrainer(
    model = AutoModelForCausalLM.from_pretrained(
        model_id,
        quantization_config=bnb_config,
        device_map={"": 0}# use the gpu
    ),
    train_dataset = train_dataset_tokenized,
    eval_dataset = eval_dataset_tokenized,
    peft_config = peft_config,  # Use our LoRA config
    max_seq_length=None,      # no max sequence length
    args=training_arguments,  # Use the training arguments
    callbacks=[early_stopping_callback],  # Add EarlyStoppingCallback
    #dataset_text_field="text", # Adjust if needed, use only if dataset not tokenized
    #max_seq_length=512,       # Set max sequence length
    #tokenizer=tokenizer,      # Use the tokenizer
    #packing=False,            # Disable packing, Packing refers to the process of grouping shorter sequences into a single batch
    #processing_class=AutoTokenizer, TODO
)

# Train the model
print('Train model...')
trainer.train()

# Save trained model
trainer.model.save_pretrained(new_model)

tokenizer_config.json:   0%|          | 0.00/55.4k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/9.09M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/296 [00:00<?, ?B/s]

Load dataset...


README.md:   0%|          | 0.00/743 [00:00<?, ?B/s]

...

generation_config.json:   0%|          | 0.00/184 [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1568: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1568: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/trl/trainer/sft_trainer.py:309: UserWarning: You didn't pass a `max_seq_length` argument to the SFTTrainer, this will default to 1024
  warnings.warn(


Train model...


Step,Training Loss,Validation Loss
10,3.012900,2.966462
20,3.007000,2.925933
30,2.953200,2.842867
40,2.650900,2.698435
50,2.553800,2.472106
60,2.337100,2.163994
70,1.915300,1.832453
80,1.742800,1.562719
90,1.328000,1.339063
100,1.325900,1.139516


Trainer.tokenizer is now deprecated. You should use Trainer.processing_class instead.
Trainer.tokenizer is now deprecated. You should use Trainer.processing_class instead.
Trainer.tokenizer is now deprecated. You should use Trainer.processing_class instead.
Trainer.tokenizer is now deprecated. You should use Trainer.processing_class instead.
Trainer.tokenizer is now deprecated. You should use Trainer.processing_class instead.
Trainer.tokenizer is now deprecated. You should use Trainer.processing_class instead.
Trainer.tokenizer is now deprecated. You should use Trainer.processing_class instead.
Trainer.tokenizer is now deprecated. You should use Trainer.processing_class instead.
Trainer.tokenizer is now deprecated. You should use Trainer.processing_class instead.
Trainer.tokenizer is now deprecated. You should use Trainer.processing_class instead.
Trainer.tokenizer is now deprecated. You should use Trainer.processing_class instead.
Trainer.tokenizer is now deprecated. You should use Tr

In [10]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


#Get last checkpoint

In [ ]:
from transformers import Trainer, TrainingArguments

# Check if a checkpoint exists in the output directory
import os

last_checkpoint = None
if os.path.isdir(output_dir):
    checkpoints = [f for f in os.listdir(output_dir) if f.startswith("checkpoint")]
    if checkpoints:
        last_checkpoint = os.path.join(output_dir, sorted(checkpoints, key=lambda x: int(x.split("-")[1]))[-1])
        print(f"Resuming from checkpoint: {last_checkpoint}")




Resuming from checkpoint: ./results/checkpoint-224
cp: cannot stat 'last_checkpoint': No such file or directory


In [11]:
!cp -r /content/results/checkpoint-200 /content/drive/MyDrive

#Merge the model

In [6]:
# Reload model in FP16 and merge it with LoRA weights
base_model = AutoModelForCausalLM.from_pretrained(
    model_id,
    low_cpu_mem_usage=True,
    return_dict=True,
    torch_dtype=torch.bfloat16,
    device_map={"": 0},
)
model = PeftModel.from_pretrained(base_model, new_model)
model = model.merge_and_unload()

# Reload tokenizer to save it
# Initialize the tokenizer with specific adjustments
# tokenizer = AutoTokenizer.from_pretrained(model_id)
# tokenizer.padding_side = 'right'
# tokenizer.pad_token = tokenizer.eos_token
#tokenizer.chat_template = get_llama3_chat_template()

Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

#Save to Hugging face

In [7]:
#model.push_to_hub("llama-3-8b-calvinscale_yes", use_temp_dir=False)
#tokenizer.push_to_hub("llama-3-8b-calvinscale_yes", use_temp_dir=False)

model.push_to_hub(new_model, use_temp_dir=False)
tokenizer.push_to_hub(new_model, use_temp_dir=False)

README.md:   0%|          | 0.00/5.17k [00:00<?, ?B/s]

model-00004-of-00004.safetensors:   0%|          | 0.00/1.17G [00:00<?, ?B/s]

model-00001-of-00004.safetensors:   0%|          | 0.00/4.98G [00:00<?, ?B/s]

model-00002-of-00004.safetensors:   0%|          | 0.00/5.00G [00:00<?, ?B/s]

model-00003-of-00004.safetensors:   0%|          | 0.00/4.92G [00:00<?, ?B/s]

Upload 4 LFS files:   0%|          | 0/4 [00:00<?, ?it/s]

tokenizer.json:   0%|          | 0.00/17.2M [00:00<?, ?B/s]

CommitInfo(commit_url='https://huggingface.co/pajacques/llama-3-8b-calvinscale_yes/commit/d51968a727e3074d842beeeb1ecddff066681956', commit_message='Upload tokenizer', commit_description='', oid='d51968a727e3074d842beeeb1ecddff066681956', pr_url=None, repo_url=RepoUrl('https://huggingface.co/pajacques/llama-3-8b-calvinscale_yes', endpoint='https://huggingface.co', repo_type='model', repo_id='pajacques/llama-3-8b-calvinscale_yes'), pr_revision=None, pr_num=None)

#Continue loading model

In [ ]:
from datasets import load_dataset
from transformers import EarlyStoppingCallback
import torch
from transformers import (
    AutoModelForCausalLM,
    AutoTokenizer,
    BitsAndBytesConfig,
    HfArgumentParser,
    TrainingArguments,
    pipeline,
    logging,
)
from datasets import Dataset
from peft import LoraConfig, PeftModel
from trl import SFTTrainer

# Load LoRA configuration
peft_config = LoraConfig(
    lora_alpha=16,
    lora_dropout=0.1,
    r=64,
    bias="none",
    task_type="CAUSAL_LM",
)

model_id = last_checkpoint

# Load the dataset
dataset = load_dataset(dataset_name)

# Load the dataset
#dataset = Dataset.from_csv("path_to_dataset.csv")  # Replace with your dataset file path

# Define the columns to use
input_column = "text"   # Column name for the input
target_column = "json"  # Column name for the target

# Load tokenizer
tokenizer = AutoTokenizer.from_pretrained(model_id)  # Replace with your model name
tokenizer.padding_side = 'right'
tokenizer.pad_token = tokenizer.eos_token

# Preprocess the dataset
def preprocess_function(examples):
    inputs = examples[input_column]  # Extract the input column
    targets = examples[target_column]  # Extract the target column
    model_inputs = tokenizer(inputs, max_length=2048, truncation=True, padding="max_length")

    # Tokenize the target and add it as labels
    labels = tokenizer(targets, max_length=2048, truncation=True, padding="max_length")
    model_inputs["labels"] = labels["input_ids"]
    return model_inputs

# Tokenize the dataset
tokenized_dataset = dataset.map(preprocess_function, batched=True)

# Calculate split indices
dataset_size = len(dataset)
train_size = int(0.8 * dataset_size)  # 80% for training
test_size = dataset_size - train_size  # 20% for testing

# Shuffle the dataset and split into training and testing sets
train_dataset = load_dataset(dataset_name, split='train')
test_dataset = load_dataset(dataset_name,  split='test')
eval_dataset = load_dataset(dataset_name,  split='validation')

# Set training parameters
training_arguments = TrainingArguments(
    output_dir=output_dir,  # Directory to save model checkpoints
    num_train_epochs=num_train_epochs,   # Set the number of epochs as max_steps=-1
    per_device_train_batch_size=2,  # Set batch size
    gradient_accumulation_steps=2,  # Gradient accumulation steps
    optim="paged_adamw_32bit",  # Optimizer
    save_steps=10,  # Save model every 10 steps
    logging_steps=5,  # Log every 10 steps
    learning_rate=2e-4,  # Learning rate
    weight_decay=0.001,  # Weight decay
    fp16=False,  # Set to True if using FP16
    bf16=True,  # Set to True for A100 GPUs
    max_grad_norm=0.3,  # Max gradient norm
    max_steps=100, # Set to -1 to disable max steps
    warmup_ratio=0.03,  # Warmup ratio
    group_by_length=True,         # Group sequences by length
    lr_scheduler_type="cosine",   # Learning rate scheduler
    report_to="tensorboard",      # Report to TensorBoard
    evaluation_strategy="steps",  # Evaluate every few steps
    eval_steps=10,                # Evaluation steps
    save_total_limit=2,           # Limit the number of saved models
    load_best_model_at_end=True,  # Load the best model at the end of training
    metric_for_best_model="loss",  # Use [loss|accuracy] to choose the best model
)

import torch
from transformers import (
    AutoTokenizer,
    AutoModelForCausalLM,
    BitsAndBytesConfig,
    pipeline,
    logging,
)

# load the quantized settings, we're doing 4 bit quantization
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.bfloat16,
    bnb_4bit_use_double_quant=False,
)

# Define EarlyStoppingCallback with patience of 3 steps
early_stopping_callback =  EarlyStoppingCallback(
    early_stopping_patience=2,    # Stop after 2 evaluation steps if no improvement
    early_stopping_threshold=0.01 # Only consider stopping if improvement is less than this threshold
)

# Set supervised fine-tuning parameters
trainer = SFTTrainer(
    #model=AutoModelForCausalLM.from_pretrained(model_id),  # Replace with your model name
    model = AutoModelForCausalLM.from_pretrained(
        model_id,
        quantization_config=bnb_config,
        # use the gpu
        device_map={"": 0}
    ),
    train_dataset=train_dataset,
    eval_dataset=eval_dataset,
    peft_config=peft_config,  # Use our LoRA config
    dataset_text_field="text", # Adjust if needed
    #max_seq_length=512,       # Set max sequence length
    max_seq_length=None,                    # no max sequence length
    tokenizer=tokenizer,      # Use the tokenizer
    args=training_arguments,  # Use the training arguments
    packing=False,            # Disable packing
    callbacks=[early_stopping_callback],  # Add EarlyStoppingCallback
    #processing_class=AutoTokenizer, TODO
)

# Train the model
trainer.train(resume_from_checkpoint=last_checkpoint)

# Save the best model
# trainer.save_model("path_to_save_best_model")  # Replace with your desired path

# Save trained model
trainer.model.save_pretrained(new_model)

Map:   0%|          | 0/4745 [00:00<?, ? examples/s]

Map:   0%|          | 0/1566 [00:00<?, ? examples/s]

Map:   0%|          | 0/1689 [00:00<?, ? examples/s]

/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1568: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_deprecation.py:100: FutureWarning: Deprecated argument(s) used in '__init__': dataset_text_field. Will not be supported from version '0.13.0'.

Deprecated positional argument(s) used in SFTTrainer, please use the SFTConfig to set these arguments instead.
  warnings.warn(message, FutureWarning)
/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1568: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1568: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/trl/trainer/sft_trainer.py:309: UserWarning: You didn't pass a `max_seq_length` argument to the SFTTrainer, this will default to 1024
  warnings.war

Map:   0%|          | 0/4745 [00:00<?, ? examples/s]

Map:   0%|          | 0/1566 [00:00<?, ? examples/s]

max_steps is given, it will override any value given in num_train_epochs
/usr/local/lib/python3.10/dist-packages/transformers/trainer.py:3354: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experime

Step,Training Loss,Validation Loss


#Fine tunning 4

In [ ]:
from transformers import T5ForConditionalGeneration, T5Tokenizer
from datasets import Dataset

# Define the tokenizer and model
tokenizer = T5Tokenizer.from_pretrained('t5-base')
model = T5ForConditionalGeneration.from_pretrained('t5-base')

# Example dataset with 'text' as the invoice and 'json' as the output
data = {
    'text': [
        "Invoice Number: 3339047245, PO Number: 300182, Vendor: PRECISION BRAND PRODUCTS, INC.",
        # Add other invoice texts here
    ],
    'json': [
        '{"invoice_number": "3339047245", "po_number": "300182", "vendor_name": "PRECISION BRAND PRODUCTS, INC."}',
        # Add other corresponding JSON outputs here
    ]
}

# Load dataset into Hugging Face Dataset
dataset = Dataset.from_dict(data)

# Tokenize and encode dataset for fine-tuning
def tokenize_function(examples):
    inputs = examples['text']
    targets = examples['json']

    model_inputs = tokenizer(inputs, padding="max_length", truncation=True, max_length=512)
    labels = tokenizer(targets, padding="max_length", truncation=True, max_length=512)

    model_inputs['labels'] = labels['input_ids']
    return model_inputs

# Tokenize the dataset
tokenized_datasets = dataset.map(tokenize_function, batched=True)

# Fine-tune the model
from transformers import Trainer, TrainingArguments

training_args = TrainingArguments(
    output_dir='./results',
    evaluation_strategy='epoch',
    learning_rate=2e-5,
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    num_train_epochs=3,
    weight_decay=0.01,
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_datasets,
    eval_dataset=tokenized_datasets,
)

trainer.train()


spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.39M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/1.21k [00:00<?, ?B/s]

You are using the default legacy behaviour of the <class 'transformers.models.t5.tokenization_t5.T5Tokenizer'>. This is expected, and simply means that the `legacy` (previous) behavior will be used so nothing changes for you. If you want to use the new behaviour, set `legacy=False`. This should only be set if you understand what it means, and thoroughly read the reason why this was added as explained in https://github.com/huggingface/transformers/pull/24565


model.safetensors:   0%|          | 0.00/892M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/147 [00:00<?, ?B/s]

Map:   0%|          | 0/1 [00:00<?, ? examples/s]

wandb: WARNING The `run_name` is currently set to the same value as `TrainingArguments.output_dir`. If this was not intended, please specify a different run name by setting the `TrainingArguments.run_name` parameter.
wandb: Using wandb-core as the SDK backend.  Please refer to https://wandb.me/wandb-core for more information.


<IPython.core.display.Javascript object>

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit:

Dataset Preparation

1. Install Required Libraries

In [ ]:
pip install transformers datasets torch


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 480.6/480.6 kB 3.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 8.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.3/179.3 kB 7.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 6.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 9.6 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2024.10.0
    Uninstalling fsspec-2024.10.0:
      Successfully uninstalled fsspec-2024.10.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2024.10.0 requires fsspec==2024.10.0, but you have fsspec 2024.9.0 which is incompatible.


2. Dataset Preparation

In [ ]:
import pandas as pd

# Example dataset
data = [
    {
        "text": "Invoice Number: 3339047245, PO Number: 300182.",
        "json": [
            {"entity": "INVOICE_NUMBER", "start": 16, "end": 26},
            {"entity": "PO_NUMBER", "start": 38, "end": 44},
        ],
    }
]

df = pd.DataFrame(data)


3. Tokenization and Annotation Alignment

In [ ]:
from transformers import AutoTokenizer
import numpy as np

# Load tokenizer
tokenizer = AutoTokenizer.from_pretrained("bert-base-uncased")

# Helper function to align labels
def align_labels_with_tokens(text, annotations, tokenizer):
    tokens = tokenizer(text, truncation=True, padding="max_length", return_offsets_mapping=True)
    offset_mapping = tokens.pop("offset_mapping")
    labels = [0] * len(tokens["input_ids"])  # Default label: 0 (no entity)

    # Define label mapping
    label_to_id = {"O": 0, "B-INVOICE_NUMBER": 1, "I-INVOICE_NUMBER": 2,
                   "B-PO_NUMBER": 3, "I-PO_NUMBER": 4}

    # Assign labels based on annotations
    for annotation in annotations:
        start, end, entity = annotation["start"], annotation["end"], annotation["entity"]
        label_prefix = f"B-{entity}"
        inside_label_prefix = f"I-{entity}"

        for idx, (token_start, token_end) in enumerate(offset_mapping):
            if token_start >= start and token_end <= end:
                if token_start == start:
                    labels[idx] = label_to_id[label_prefix]
                else:
                    labels[idx] = label_to_id[inside_label_prefix]

    tokens["labels"] = labels
    return tokens

# Apply tokenization and alignment to the dataset
processed_data = [
    align_labels_with_tokens(row["text"], row["json"], tokenizer) for _, row in df.iterrows()
]

# Example processed data
print(processed_data[0])


tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

{'input_ids': [101, 1999, 6767, 6610, 2193, 1024, 21211, 21057, 22610, 18827, 2629, 1010, 13433, 2193, 1024, 3998, 15136, 2475, 1012, 102, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 

4. Prepare Dataset for Training

In [ ]:
from torch.utils.data import Dataset

class NERDataset(Dataset):
    def __init__(self, data):
        self.data = data

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        item = self.data[idx]
        return {key: np.array(value) for key, value in item.items()}

# Create dataset object
train_dataset = NERDataset(processed_data)


5. Define the Model and Trainer

In [ ]:
from transformers import AutoModelForTokenClassification, TrainingArguments, Trainer
from sklearn.model_selection import train_test_split

# Split the processed data into train and eval datasets
train_data, eval_data = train_test_split(processed_data, test_size=0.2, random_state=42)

# Create dataset objects for train and eval
train_dataset = NERDataset(train_data)
eval_dataset = NERDataset(eval_data)


# Load pre-trained model
model = AutoModelForTokenClassification.from_pretrained(
    "bert-base-uncased",
    num_labels=5  # Number of entity labels
)

# Define training arguments
training_args = TrainingArguments(
    output_dir="./results",
    evaluation_strategy="epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=16,
    num_train_epochs=3,
    weight_decay=0.01,
    logging_dir="./logs",
)

# Define trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=eval_dataset,  # Include eval dataset for evaluation
    tokenizer=tokenizer,
)


ValueError: With n_samples=1, test_size=0.2 and train_size=None, the resulting train set will be empty. Adjust any of the aforementioned parameters.

6. Train the Model

In [ ]:
trainer.train()


### Run the model (after the Fine Tuned Model)
The following allows you to test your own fine tuned model

###Run -1

In [ ]:
# Execute the query
pipe = pipeline(task="text-generation", model=model, tokenizer=tokenizer)

# Get the prompt
prompt = get_prompt('/content/Facture client_9372_20240703_153551.pdf')

# Generate the result
result = pipe(f"{prompt}", temperature=0.01)

# Show the result by removing the input token (prompt) from the generated text
generated_text = result[0]['generated_text']

# Remove the prompt portion from the generated text and print only the answer
answer = generated_text[len(prompt):].strip()
print(answer)

### Run-2

In [ ]:
!pip install --quiet PyMuPDF

NotImplementedError: A UTF-8 locale is required. Got ANSI_X3.4-1968

In [ ]:
# import sys, pathlib, fitz, os

# import torch
# from transformers import pipeline

# def extract_text_from_pdf3(text):

#     # Initialize the pipeline
#     pipe = pipeline(task="text-generation", model=model, tokenizer=tokenizer)

#     # Get the prompt
#     prompt = get_prompt_from_text(text)

#     # Tokenize the input text
#     tokens = tokenizer(prompt, return_tensors="pt")

#     # Count the number of tokens
#     num_tokens = len(tokens['input_ids'][0])
#     print('Number of token:', num_tokens)
#     if num_tokens > 4000:
#       return 'TOO MUCH DATA'

#     # Use mixed-precision (AMP) during the generation
#     #with torch.cuda.amp.autocast(device_type='cuda', enabled=False, dtype=torch.bfloat16):
#     with torch.cuda.amp.autocast():
#         # Generate the result
#         result = pipe(f"{prompt}", temperature=0.01)

#     # Show the result by removing the input token (prompt) from the generated text
#     generated_text = result[0]['generated_text']

#     # Remove the prompt portion from the generated text and print only the answer
#     answer = generated_text[len(prompt):].strip()
#     return answer


# def extract_text_from_pdf1(text):
#   print('ALLOOooooooooooooo')

#   # Execute the query
#   pipe = pipeline(task="text-generation", model=model, tokenizer=tokenizer)

#   # Get the prompt
#   prompt = get_prompt_from_text(text)

#   # Tokenize the input text
#   tokens = tokenizer(prompt, return_tensors="pt")

#   # Count the number of tokens
#   num_tokens = len(tokens['input_ids'][0])

#   print('*****token:',num_tokens)


#   # Generate the result
#   result = pipe(f"{prompt}", temperature=0.01)

#   # Show the result by removing the input token (prompt) from the generated text
#   generated_text = result[0]['generated_text']

#   # Remove the prompt portion from the generated text and print only the answer
#   answer = generated_text[len(prompt):].strip()
#   print(answer)
#   print('*****************')

# fname = '/content/Facture client_9372_20240703_153551.pdf'
# fname = '/content/SCN Industrial (1).pdf'
# fname = '/content/Makita - GROUPE LD INC.pdf'
# fname = '/content/PSAProjInvoice.Report (5).pdf'
# fname = '/content/3M Facture 437549 – 9425384536.PDF'
# words=""
# with fitz.open(fname) as pages:
#   for i, page in enumerate(pages):
#     words = page.get_text("text", sort=True)
#     print(f'**** PAGE {i+1} ****')
#     print(words)
#     print(f'****END OF PAGE {i+1} ****')
#     print(f'**** BEGIN PREDICTION PAGE {i+1} ****')
#     r = extract_text_from_pdf3(words)
#     print(r)
#     print(f'**** END PREDICTION PAGE {i+1} ****')


In [ ]:
# import torch
# from transformers import pipeline

# import torch
# from transformers import AutoModelForCausalLM, AutoTokenizer

# def extract_text_from_pdf2(text):
#     # Load the model and tokenizer
#     # model = AutoModelForCausalLM.from_pretrained(model).to(torch.bfloat16)  # Set model to BFloat16
#     # tokenizer = tokenizer#AutoTokenizer.from_pretrained("model_name")

#     # Get the prompt
#     prompt = get_prompt_from_text(text)

#     # Tokenize the input text
#     tokens = tokenizer(prompt, return_tensors="pt").to(torch.bfloat16)  # Convert to BFloat16

#     # Count the number of tokens
#     num_tokens = len(tokens['input_ids'][0])
#     print('*****token:', num_tokens)

#     # Generate the result using the model
#     with torch.no_grad():  # Disable gradient calculation
#         output = model.generate(**tokens, temperature=0.01)

#     # Decode the generated output
#     generated_text = tokenizer.decode(output[0], skip_special_tokens=True)
#     print(generated_text)

# # Example PDF extraction logic remains the same


# fname = '/content/Facture client_9372_20240703_153551.pdf'
# words = ""
# with fitz.open(fname) as pages:
#     for page in pages:
#         words = page.get_text("text", sort=True)
#         print(f'* Extracting from text {words}')
#         extract_text_from_pdf2(words)


## Clear the Model
The following will clear the model from memory

In [ ]:
print(answer)

In [ ]:
# Empty VRAM
del model
del pipe
del trainer

# clear memory
import torch
torch.cuda.empty_cache()

# garbage collect
import gc
gc.collect()
gc.collect()

NameError: name 'pipe' is not defined

Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

In [ ]:
#model.push_to_hub("llama-3-8b-calvinscale_yes", use_temp_dir=False)
#tokenizer.push_to_hub("llama-3-8b-calvinscale_yes", use_temp_dir=False)

model.push_to_hub(new_model, use_temp_dir=False)
tokenizer.push_to_hub(new_model, use_temp_dir=False)

README.md:   0%|          | 0.00/5.17k [00:00<?, ?B/s]

model-00003-of-00004.safetensors:   0%|          | 0.00/4.92G [00:00<?, ?B/s]

model-00002-of-00004.safetensors:   0%|          | 0.00/5.00G [00:00<?, ?B/s]

model-00001-of-00004.safetensors:   0%|          | 0.00/4.98G [00:00<?, ?B/s]

Upload 3 LFS files:   0%|          | 0/3 [00:00<?, ?it/s]

No files have been modified since last commit. Skipping to prevent empty commit.


CommitInfo(commit_url='https://huggingface.co/pajacques/llama-3-8b-calvinscale_yes/commit/ab2d1fd015fd656c4979e06d17d716f8058746b0', commit_message='Upload tokenizer', commit_description='', oid='ab2d1fd015fd656c4979e06d17d716f8058746b0', pr_url=None, repo_url=RepoUrl('https://huggingface.co/pajacques/llama-3-8b-calvinscale_yes', endpoint='https://huggingface.co', repo_type='model', repo_id='pajacques/llama-3-8b-calvinscale_yes'), pr_revision=None, pr_num=None)

In [ ]:
from google.colab import runtime
runtime.unassign()

#Test the model

In [ ]:
!pip uninstall pyarrow -y
!pip install --quiet pyarrow==15.0.0
!pip install --quiet datasets
!pip install -U bitsandbytes

Found existing installation: pyarrow 17.0.0
Uninstalling pyarrow-17.0.0:
  Successfully uninstalled pyarrow-17.0.0
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 38.3/38.3 MB 57.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 480.6/480.6 kB 10.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 9.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.3/179.3 kB 13.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 10.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 16.7 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2024.10.0 requires fsspec==2024.10.0, but you have fsspec 2024.9.0 which is incompatible.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 122.4/122.4 MB 12.3 MB/s eta 0:00:00


In [ ]:
!pip install transformers
!pip install datasets Pillow

!pip install --quiet pymupdf

# install dependencies

# we use the latest version of transformers, peft, and accelerate
!pip install --quiet -q accelerate peft transformers

# install bitsandbytes for quantization
!pip install --quiet -q bitsandbytes
#!pip install -U bitsandbytes

# install trl for the SFT library
!pip install --quiet -q trl

# we need to install datasets for our training dataset
!pip install --quiet -q datasets

# we need huggingface hub to get access to the llama 3 model
!pip install --quiet huggingface_hub

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 19.8/19.8 MB 92.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 316.6/316.6 kB 6.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 109.8/109.8 kB 12.7 MB/s eta 0:00:00


##Connect to hugging face

In [ ]:
from huggingface_hub import login
from google.colab import userdata

token = userdata.get('HF_TOKEN')

# Log in to Hugging Face
login(token, add_to_git_credential=True)

Token is valid (permission: fineGrained).
Your token has been saved in your configured git credential helpers (store).
Your token has been saved to /root/.cache/huggingface/token
Login successful


##Inference #1 using HuggingFace

In [ ]:
!pip install -U bitsandbytes
!pip install accelerate

In [ ]:
!pip install --upgrade pip

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 59.0 MB/s eta 0:00:00
  Attempting uninstall: pip
    Found existing installation: pip 24.1.2
    Uninstalling pip-24.1.2:
      Successfully uninstalled pip-24.1.2


In [ ]:
#!pip uninstall  bitsandbytes
!pip install -U bitsandbytes

In [ ]:
import torch
from transformers import (
    AutoModelForCausalLM,
    AutoTokenizer,
    BitsAndBytesConfig,
    HfArgumentParser,
    TrainingArguments,
    pipeline,
    logging,
)
from peft import LoraConfig, PeftModel
from trl import SFTTrainer

# we will use llama-3 instruct as our base model
#model_id = "pajacques/llama-3-8b-calvinscale"
model_id = "pajacques/llama-3-8b-calvinscale_yes"

tokenizer = AutoTokenizer.from_pretrained(model_id)
tokenizer.padding_side = 'right'
tokenizer.pad_token = tokenizer.eos_token

import torch
from transformers import (
    AutoTokenizer,
    AutoModelForCausalLM,
    BitsAndBytesConfig,
    pipeline,
    logging,
)

# load the quantized settings, we're doing 4 bit quantization
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.bfloat16,
    bnb_4bit_use_double_quant=False,
)

# Load base model
model = AutoModelForCausalLM.from_pretrained(
    model_id,
    quantization_config=bnb_config,
    # use the gpu
    device_map={"": 0}
)

# don't use the cache
model.config.use_cache = False

import sys, pathlib, fitz
def getText(fname):
    all_text = ""  # Initialize an empty string to store the concatenated text
    with fitz.open(fname) as pages:
        for page in pages:
            words = page.get_text("text", sort=True)  # Get the text of the current page
            all_text += words  # Append the text of the page to the 'all_text' variable
    return all_text

def get_prompt(fname):
  with open("/content/prompt_template.txt", "r") as file:
    template = file.read()
  template = template.replace("#####", getText(fname))
  return template

prompt = get_prompt('/content/Facture client_9372_20240703_153551.pdf')
#prompt = "How hot is 7 degrees Calvin on the Calvin Scale?"
#prompt = "What is the capital of canada?"

# execute the query
pipe = pipeline(task="text-generation", model=model, tokenizer=tokenizer)
result = pipe(f"{prompt}", temperature=0.1)

# show the result
print(result[0]['generated_text'])

ImportError: Using `bitsandbytes` 4-bit quantization requires the latest version of bitsandbytes: `pip install -U bitsandbytes`

##Inference #2 using PyTorch

In [ ]:
import sys, pathlib, fitz
from transformers import AutoModelForCausalLM, AutoTokenizer
import torch

# Load model and tokenizer
model_name = "pajacques/llama-3-8b-calvinscale_yes"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForCausalLM.from_pretrained(model_name).to("cuda")

def getText(fname):
    all_text = ""  # Initialize an empty string to store the concatenated text
    with fitz.open(fname) as pages:
        for page in pages:
            words = page.get_text("text", sort=True)  # Get the text of the current page
            all_text += words  # Append the text of the page to the 'all_text' variable
    return all_text

def get_prompt(fname):
    with open("/content/prompt_template.txt", "r") as file:
      template = file.read()
    template = template.replace("#####", getText(fname))
    return template

# Function for fast inference
def generate_text(prompt):
    inputs = tokenizer(prompt, return_tensors="pt", padding=True, truncation=True).to("cuda")

    with torch.cuda.amp.autocast():  # Use mixed precision
        outputs = model.generate(**inputs, max_new_tokens=150)  # Set appropriate max tokens

    return tokenizer.decode(outputs[0], skip_special_tokens=True)

tokenizer_config.json:   0%|          | 0.00/51.0k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/9.09M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/325 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/728 [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/23.9k [00:00<?, ?B/s]

...

model-00002-of-00004.safetensors:   0%|          | 0.00/5.00G [00:00<?, ?B/s]

model-00003-of-00004.safetensors:   0%|          | 0.00/4.92G [00:00<?, ?B/s]

model-00004-of-00004.safetensors:   0%|          | 0.00/1.17G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/194 [00:00<?, ?B/s]

##Ask the model

In [ ]:
# Example usage
prompt = get_prompt('/content/Facture client_9372_20240703_153551.pdf')
response = generate_text(prompt)
print(response)

In [ ]:
!cp -r /content/results/checkpoint-193 /content/drive/MyDrive


In [ ]:
from transformers import BigBirdTokenizer, BigBirdForSequenceClassification, Trainer, TrainingArguments
from datasets import Dataset

# Load the tokenizer
tokenizer = BigBirdTokenizer.from_pretrained('google/bigbird-base-trivia-itc')

# Prepare a sample dataset
data = [
    {"text": "Invoice number: 3339047245. PO number: 300182. Vendor: PRECISION BRAND PRODUCTS, INC.", "label": 1},  # Invoice
    {"text": "Purchase order 300183 includes invoice number 1234567890 for vendor ABC.", "label": 2}  # Purchase Order
]

# Convert the dataset to Hugging Face Dataset format
dataset = Dataset.from_dict({
    "text": [item["text"] for item in data],
    "label": [item["label"] for item in data]
})

# Tokenize the text
def tokenize_function(examples):
    return tokenizer(examples['text'], padding="max_length", truncation=True, max_length=512)

tokenized_dataset = dataset.map(tokenize_function, batched=True)

# Load the model for sequence classification
model = BigBirdForSequenceClassification.from_pretrained('google/bigbird-base-trivia-itc', num_labels=3)

# Training arguments
training_args = TrainingArguments(
    output_dir="./results",
    evaluation_strategy="epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=16,
    num_train_epochs=3,
    weight_decay=0.01,
    logging_dir="./logs",
)

# Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_dataset,
    eval_dataset=tokenized_dataset,
)

# Train the model
trainer.train()

# Save the trained model
trainer.save_model("./trained_bigbird_sequence_model")


tokenizer_config.json:   0%|          | 0.00/969 [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/846k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/775 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/789 [00:00<?, ?B/s]

Map:   0%|          | 0/2 [00:00<?, ? examples/s]

...

 ··········


wandb: ERROR API key must be 40 characters long, yours was 37


<IPython.core.display.Javascript object>

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit:


Abort: 

In [ ]:
from transformers import BigBirdForQuestionAnswering, BigBirdTokenizer
import torch

# Load the BigBird tokenizer and model
model_name = "google/bigbird-base-trivia-itc"
tokenizer = BigBirdTokenizer.from_pretrained(model_name)
model = BigBirdForQuestionAnswering.from_pretrained(model_name)

# Define question and context
question = "What is the po number from this invoice text PRECISION BRAND PRODUCTS, INC. INVOICE A Marmon/Berkshire Hathaway Company Invoice Number: 6047629942 REMIT IN CANADIAN FUNDS TO: (630) 969-7200 www.precisionbrand.com PRECISION BRAND PRODUCTS, INC. Page: 1 BOX T57412C Date: 9/11/2024 Corporate office address: PO BOX 57412 STATION A 2250 Curtiss Street TORONTO ON M5W 5M5 CANADA Salesperson: 2 67 AES 2QC Downers Grove, IL 60515 USA Customer: 1 65 51464 DUNS 09-646-0860 Sales Order: 0000104000 Sold To Ship To Groupe Ld Inc Industrie LD Distribution Dmc Inc 1250 Rte Marie Victorin 2370 Rue Bauman Sorel Tracy, QC J3R 1L9 CAN Jonquiere, QC G7S 4S4 CAN Customer P.O. Ship Via F.O.B Terms 691960 Purolator 2% Disc Ten Days Due in 30 Item Description Qty Shipped Price Amount 51703 571 X 81 X 4 KYMZB WISQ - 0/N 5.1 PKG 30.4393 155.24 27336 732 X 62 X 1 WNIRQ ALHM - 5/F 7.0 PKG 16.8001 117.60 11378 263 X 46 X 4 FCPSZ MITD - 3/Z 2.2 PKG 15.6764 34.49 42937 350 X 04 X 0 JKXQS PKRH - 9/M 6.8 PKG 16.4196 111.65 65123 194 X 52 X 7 OCGAZ CUVF - 7/Y 6.3 PKG 26.8173 168.95 35303 806 X 51 X 4 ROKXC UGLU - 6/N 3.5 PKG 54.0902 189.32 LAST ITEM Subtotal 559.87 Freight 0.00 GST/HST 31.54 Trade Discount 0.00 Payment/Credit Amount 0.00 Additional Charges 0.00 Contact: Primary Contact Balance CAD 592.96 We hereby certify that these goods were produced in compliance with all applicable requirements of Section 6, 7, and 12 of the Fair Labor Standards Act, as amended, and of regulations and orders of the United States Department of Labor issued under Section 14, thereof. CONTINGENT LIABILITY We guarantee to replace any material that is defective within 10 days, provided that it has been promptly worked or used. Such material will be replaced or credit will be allowed for the price thereof upon its return, but no material may be returned except upon our authorization. If any material proves defective or not in conformity with your specifications, your sole remedy will be to secure replacement of such material. Under no circumstances will we be liable for any claims for labor or consequential damage. PBP 1200E R02//21 Page 1 of 1"
context = '[ { "end": 3655, "entity": "INVOICE_NUMBER", "start": 3645 }, { "end": 5004, "entity": "PO_NUMBER", "start": 4998 }, { "end": 3437, "entity": "VENDOR_NAME", "start": 3407 }, { "end": 4310, "entity": "CUSTOMER_NO", "start": 4300 }, { "end": 6518, "entity": "GST_TAX_AMOUNT", "start": 6513 }, { "end": 6276, "entity": "SUB_TOTAL", "start": 6270 }, { "end": 7028, "entity": "INVOICE_TOTAL", "start": 7022 }, { "end": 7007, "entity": "CURRENCY", "start": 7004 }, { "end": 5247, "entity": "ITEM_CODE", "start": 5242 }, { "end": 5295, "entity": "ITEM_NAME", "start": 5266 }, { "end": 5332, "entity": "QUANTITY", "start": 5329 }, { "end": 5356, "entity": "UNIT_PRICE", "start": 5349 }, { "end": 5379, "entity": "LINE_TOTAL", "start": 5373 }, { "end": 5386, "entity": "ITEM_CODE", "start": 5381 }, { "end": 5434, "entity": "ITEM_NAME", "start": 5405 }, { "end": 5471, "entity": "QUANTITY", "start": 5468 }, { "end": 5496, "entity": "UNIT_PRICE", "start": 5489 }, { "end": 5519, "entity": "LINE_TOTAL", "start": 5513 }, { "end": 5526, "entity": "ITEM_CODE", "start": 5521 }, { "end": 5574, "entity": "ITEM_NAME", "start": 5545 }, { "end": 5611, "entity": "QUANTITY", "start": 5608 }, { "end": 5635, "entity": "UNIT_PRICE", "start": 5628 }, { "end": 5657, "entity": "LINE_TOTAL", "start": 5652 }, { "end": 5664, "entity": "ITEM_CODE", "start": 5659 }, { "end": 5712, "entity": "ITEM_NAME", "start": 5683 }, { "end": 5493, "entity": "QUANTITY", "start": 5490 }, { "end": 5773, "entity": "UNIT_PRICE", "start": 5766 }, { "end": 5796, "entity": "LINE_TOTAL", "start": 5790 }, { "end": 5803, "entity": "ITEM_CODE", "start": 5798 }, { "end": 5851, "entity": "ITEM_NAME", "start": 5822 }, { "end": 5888, "entity": "QUANTITY", "start": 5885 }, { "end": 5912, "entity": "UNIT_PRICE", "start": 5905 }, { "end": 5935, "entity": "LINE_TOTAL", "start": 5929 }, { "end": 5942, "entity": "ITEM_CODE", "start": 5937 }, { "end": 5990, "entity": "ITEM_NAME", "start": 5961 }, { "end": 6027, "entity": "QUANTITY", "start": 6024 }, { "end": 6051, "entity": "UNIT_PRICE", "start": 6044 }, { "end": 6074, "entity": "LINE_TOTAL", "start": 6068 } ]'

# Encode the question and context
encoded_input = tokenizer(question, context, return_tensors='pt')

# Get model output
output = model(**encoded_input)

# Extract start and end logits
start_logits = output.start_logits
end_logits = output.end_logits

# Find the most probable start and end positions
start_idx = torch.argmax(start_logits)
end_idx = torch.argmax(end_logits)

# Decode the predicted answer
answer_tokens = encoded_input['input_ids'][0][start_idx:end_idx + 1]
answer = tokenizer.decode(answer_tokens, skip_special_tokens=True)

print(f"Predicted Answer: {answer}")


Input ids are automatically padded from 1493 to 1536 to be a multiple of `config.block_size`: 64


Predicted Answer: 


In [ ]:
from transformers import BigBirdTokenizer, BigBirdForSequenceClassification, Trainer, TrainingArguments
from datasets import Dataset


model_id = 'google/bigbird-roberta-large'

# Load the tokenizer
tokenizer = BigBirdTokenizer.from_pretrained(model_id)

# Prepare a sample dataset
data = [
    {"text": "Invoice number: 3339047245. PO number: 300182. Vendor: PRECISION BRAND PRODUCTS, INC.", "label": 1},  # Invoice
    {"text": "Purchase order 300183 includes invoice number 1234567890 for vendor ABC.", "label": 2}  # Purchase Order
]

# Convert the dataset to Hugging Face Dataset format
dataset = Dataset.from_dict({
    "text": [item["text"] for item in data],
    "label": [item["label"] for item in data]
})

# Tokenize the text
def tokenize_function(examples):
    return tokenizer(examples['text'], padding="max_length", truncation=True, max_length=512)

tokenized_dataset = dataset.map(tokenize_function, batched=True)

# Load the model for sequence classification
model = BigBirdForSequenceClassification.from_pretrained(model_id, num_labels=3)

# Training arguments
training_args = TrainingArguments(
    output_dir="./results",
    evaluation_strategy="epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=16,
    num_train_epochs=3,
    weight_decay=0.01,
    logging_dir="./logs",
)

# Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_dataset,
    eval_dataset=tokenized_dataset,
)

# Train the model
trainer.train()

# Save the trained model
trainer.save_model("./trained_bigbird_sequence_model")


tokenizer_config.json:   0%|          | 0.00/969 [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/846k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/775 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/758 [00:00<?, ?B/s]

Map:   0%|          | 0/2 [00:00<?, ? examples/s]

pytorch_model.bin:   0%|          | 0.00/1.44G [00:00<?, ?B/s]

Some weights of BigBirdForSequenceClassification were not initialized from the model checkpoint at google/bigbird-roberta-large and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1568: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(
Attention type 'block_sparse' is not possible if sequence_length: 512 <= num global tokens: 2 * config.block_size + min. num sliding tokens: 3 * config.block_size + config.num_random_blocks * config.block_size + additional buffer: config.num_random_blocks * config.block_size = 704 with config.block_size = 64, config.num_random_blocks = 3. Changing attention type to 'original_full'...


model.safetensors:   0%|          | 0.00/1.44G [00:00<?, ?B/s]

Epoch,Training Loss,Validation Loss
1,No log,1.016745


In [ ]:
!pip install transformers datasets torch sentencepiece

In [ ]:
from transformers import BigBirdForQuestionAnswering, BigBirdTokenizer
import torch

# Load BigBird model and tokenizer
model = BigBirdForQuestionAnswering.from_pretrained("google/bigbird-roberta-base")
tokenizer = BigBirdTokenizer.from_pretrained("google/bigbird-roberta-base")

# Sample input
question = "What is the total amount?"
context = "Invoice number: 123456789, Date: 2024-11-24, Total amount: $1500"

# Tokenize the input
encoded_input = tokenizer(question, context, return_tensors='pt', padding="max_length", truncation=True, max_length=1000)

# Get model output
output = model(**encoded_input)

# Extract start and end logits
start_logits = output.start_logits
end_logits = output.end_logits

# Get predicted start and end positions
start_idx = torch.argmax(start_logits)
end_idx = torch.argmax(end_logits)

# Decode the answer from the tokenized input
answer_tokens = encoded_input['input_ids'][0][start_idx:end_idx + 1]
answer = tokenizer.decode(answer_tokens, skip_special_tokens=True)

print(f"Predicted Answer: {answer}")


config.json:   0%|          | 0.00/760 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/513M [00:00<?, ?B/s]

Some weights of BigBirdForQuestionAnswering were not initialized from the model checkpoint at google/bigbird-roberta-base and are newly initialized: ['qa_classifier.intermediate.dense.bias', 'qa_classifier.intermediate.dense.weight', 'qa_classifier.output.LayerNorm.bias', 'qa_classifier.output.LayerNorm.weight', 'qa_classifier.output.dense.bias', 'qa_classifier.output.dense.weight', 'qa_classifier.qa_outputs.bias', 'qa_classifier.qa_outputs.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


tokenizer_config.json:   0%|          | 0.00/1.02k [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/846k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/775 [00:00<?, ?B/s]

Input ids are automatically padded from 1000 to 1024 to be a multiple of `config.block_size`: 64


model.safetensors:   0%|          | 0.00/513M [00:00<?, ?B/s]

Predicted Answer: -11-24, Total amount: $1500


In [ ]:
import torch
from datasets import Dataset
from transformers import BigBirdTokenizer, BigBirdForTokenClassification, Trainer, TrainingArguments
from transformers import DataCollatorForTokenClassification

# Step 1: Load BigBird tokenizer and model
tokenizer = BigBirdTokenizer.from_pretrained("google/bigbird-roberta-base")
model = BigBirdForTokenClassification.from_pretrained("google/bigbird-roberta-base", num_labels=2)  # for start and end positions

# Step 2: Preprocessing function
def preprocess_function(examples):
    # Tokenize the text
    encodings = tokenizer(examples['text'], truncation=True, padding='max_length', max_length=1000)

    # Create label arrays for start and end positions
    start_positions = []
    end_positions = []

    for i in range(len(examples['entities'])):
        entity = examples['entities'][0]
        start_pos = entity[i]['start']
        end_pos = entity[i]['end']

        # Map the character positions to token indices
        start_token_pos = tokenizer.convert_tokens_to_ids(tokenizer.tokenize(examples['text'][:start_pos]))
        end_token_pos = tokenizer.convert_tokens_to_ids(tokenizer.tokenize(examples['text'][:end_pos]))

        # Add to start and end position arrays
        start_positions.append(start_token_pos)
        end_positions.append(end_token_pos)

    # Return the tokenized input and labels
    encodings['start_positions'] = start_positions
    encodings['end_positions'] = end_positions
    return encodings

# Step 3: Prepare the dataset
# Assume you have a dataset in the form of a list of dictionaries
dataset = [
    {
        "text": "Invoice number: 12345, Date: 2024-11-24, Total: $1500",
        "entities": [
            {"entity": "invoice_number", "start": 15, "end": 20},
            {"entity": "date", "start": 28, "end": 38},
            {"entity": "total", "start": 41, "end": 46}
        ]
    },
    # More examples...
]

# Convert dataset to Hugging Face Dataset format
dataset = Dataset.from_dict({
    "text": [entry["text"] for entry in dataset],
    "entities": [entry["entities"] for entry in dataset]
})

# Step 4: Apply the preprocessing function
dataset = dataset.map(preprocess_function, batched=True)

# Step 5: Prepare DataCollator (handles padding dynamically)
data_collator = DataCollatorForTokenClassification(tokenizer)

# Step 6: Training arguments
training_args = TrainingArguments(
    output_dir='./results',          # output directory
    evaluation_strategy="epoch",     # evaluate after each epoch
    learning_rate=2e-5,              # learning rate
    per_device_train_batch_size=16,  # batch size for training
    per_device_eval_batch_size=16,   # batch size for evaluation
    num_train_epochs=3,              # number of training epochs
    weight_decay=0.01,               # weight decay
)

# Step 7: Trainer setup
trainer = Trainer(
    model=model,                     # the model to train
    args=training_args,              # training arguments
    data_collator=data_collator,     # data collator
    train_dataset=dataset,           # training dataset
    eval_dataset=dataset,            # evaluation dataset (can be split for validation)
)

# Step 8: Train the model
trainer.train()


Some weights of BigBirdForTokenClassification were not initialized from the model checkpoint at google/bigbird-roberta-base and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Map:   0%|          | 0/1 [00:00<?, ? examples/s]

#1 ['Invoice number: 12345, Date: 2024-11-24, Total: $1500']


TypeError: unhashable type: 'list'

In [ ]:
# from huggingface_hub import login
# from google.colab import userdata

# token = userdata.get('HF_TOKEN')

# # Log in to Hugging Face
# login(token, add_to_git_credential=True)

import wandb

wandb.login(key="your_api_key_here")

In [ ]:
!pip install --quiet datasets

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 480.6/480.6 kB 8.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 7.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.3/179.3 kB 10.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 7.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 12.3 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2024.10.0 requires fsspec==2024.10.0, but you have fsspec 2024.9.0 which is incompatible.


In [ ]:
print(data)

[{'text': 'Invoice 12345 for customer ABC Corp', 'annotations': [{'start': 8, 'end': 13, 'entity': 'INVOICE_NUMBER'}, {'start': 27, 'end': 36, 'entity': 'CUSTOMER_NAME'}]}]


In [ ]:
from transformers import RobertaTokenizerFast, RobertaForTokenClassification, Trainer, TrainingArguments
import torch
from datasets import Dataset

# Load tokenizer
tokenizer = RobertaTokenizerFast.from_pretrained("roberta-base")

# Example data
data3 = [
    {
        "text": "Invoice 12345 for customer ABC Corp",
        "annotations": [
            {"start": 8, "end": 13, "entity": "INVOICE_NUMBER"},
            {"start": 27, "end": 36, "entity": "CUSTOMER_NAME"},
        ],
    }
]

# Convert JSON to token-level labels
def preprocess_data(data, tokenizer, label_map, max_length=512):
    input_ids, attention_masks, label_ids = [], [], []
    print('data:',data)
    for item in data:
        text = item["text"]
        entities = item["annotations"]

        # Tokenize text
        encoding = tokenizer(
            text,
            return_offsets_mapping=True,
            padding="max_length",
            truncation=True,
            max_length=max_length,
        )
        offsets = encoding["offset_mapping"]
        tokens = encoding["input_ids"]

        # Initialize labels
        token_labels = ["O"] * len(offsets)

        # Assign labels based on entity spans
        for entity in entities:
            start, end, label = entity["start"], entity["end"], entity["entity"]
            print('label:',label)
            for i, (start_offset, end_offset) in enumerate(offsets):
                if start_offset >= start and end_offset <= end:
                    if start_offset == start:
                        token_labels[i] = f"B-{label}"
                    else:
                        token_labels[i] = f"I-{label}"

        # Convert labels to IDs
        label_ids.append([label_map[label] for label in token_labels])

        input_ids.append(tokens)
        attention_masks.append(encoding["attention_mask"])

    return {"input_ids": input_ids, "attention_mask": attention_masks, "labels": label_ids}


# Define label map
label_map = {
    "O": 0,
    "B-INVOICE_NUMBER": 1,
    "I-INVOICE_NUMBER": 2,
    "B-CUSTOMER_NAME": 3,
    "I-CUSTOMER_NAME": 4,
}

encoded_data = preprocess_data(data3, tokenizer, label_map)

# Convert to Hugging Face dataset
#dataset = Dataset.from_dict({"text": texts, "labels": labels})
dataset = Dataset.from_dict(encoded_data)

# Load model
model = RobertaForTokenClassification.from_pretrained("roberta-base", num_labels=len(label_map))

# Define training arguments
training_args = TrainingArguments(
    output_dir="./results",
    evaluation_strategy="epoch",
    learning_rate=5e-5,
    per_device_train_batch_size=8,
    num_train_epochs=3,
    weight_decay=0.01,
)

eval_texts, eval_labels = preprocess_data(eval_data, tokenizer, label_map)
eval_dataset = Dataset.from_dict({"text": eval_texts, "labels": eval_labels})

# Define Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=dataset,
    eval_dataset=eval_dataset,  # Add this line
    tokenizer=tokenizer,
)

# Train the model
trainer.train()


data: [{'text': 'Invoice 12345 for customer ABC Corp', 'annotations': [{'start': 8, 'end': 13, 'entity': 'INVOICE_NUMBER'}, {'start': 27, 'end': 36, 'entity': 'CUSTOMER_NAME'}]}]
label: INVOICE_NUMBER
label: CUSTOMER_NAME


Some weights of RobertaForTokenClassification were not initialized from the model checkpoint at roberta-base and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1568: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


data: [{'text': 'Another invoice text for evaluation...', 'annotations': [{'start': 30, 'end': 40, 'entity': 'CUSTOMER_NO'}, {'start': 50, 'end': 60, 'entity': 'GST_TAX_AMOUNT'}]}]
label: CUSTOMER_NO
label: GST_TAX_AMOUNT


KeyError: 'I-CUSTOMER_NO'